In [1]:
#@title Imports

#!pip install Unidecode
#from unidecode import unidecode
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import seaborn as sns
import ast
from matplotlib.lines import Line2D

from matplotlib.pyplot import *
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import scipy.stats

from sklearn import linear_model
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2
import numpy as np
import random

import scipy.stats
from scipy import stats

all_the_years = np.arange(1974, 2023, 1)
some_of_the_years = np.arange(1997, 2023, 1)
pd.set_option('display.colheader_justify', 'center')


sns.set_style('darkgrid')        # darkgrid, white grid, dark, white and ticks
plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=14)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=13)    # fontsize of the tick labels
plt.rc('ytick', labelsize=13)    # fontsize of the tick labels
plt.rc('legend', fontsize=13)    # legend fontsize
plt.rc('font', size=13)          # controls default text sizes

import re

def label_point(x, y, val, ax):
    a = pd.concat({'x': x, 'y': y, 'val': val}, axis=1)
    for i, point in a.iterrows():
      #if i % 2 == 0:
      #  ax.text(point['x']+0.25, point['y'], str(point['val']))
      #else:
      #ax.text(point['x']+0.30, point['y'], str(point['val']))
      ax.text(point['x']+0.06, point['y'], str(point['val']))

def label_point_year(x, y, val, ax):
    a = pd.concat({'x': x, 'y': y, 'val': val}, axis=1)
    for i, point in a.iterrows():
      #if i % 2 == 0:
      ax.text(point['x']+0.12, point['y'], int(point['val']))
      #else:
      #ax.text(point['x']+0.30, point['y'], str(point['val']))
      #ax.text(point['x']+0.05, point['y'], str(point['val']))


In [ ]:
#@title Import Selenium
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

In [ ]:
#@title Start Webdriver and VirtualDisplay
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

service = Service(executable_path=r'/usr/bin/chromedriver')
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome(service=service, options=options)

#options = Options()
#options.add_argument("--headless")
#options.add_argument("--no-sandbox")
#options.headless = True

#wd = webdriver.Chrome("/usr/bin/chromedriver", options=options)

!pip install pyvirtualdisplay
!apt-get install xvfb
from pyvirtualdisplay import Display
display = Display(visible=0, size=(800, 800))
display.start()

In [ ]:
#@title nba playoff ALL shooting zone scrapes (Career)

def nba_all_shooting_data(url_df):

    era_opponent_adj_playoff_per_75_df = pd.read_csv('/content/era_opponent_adjusted_per75_data.csv')

    new_df = pd.DataFrame(columns = ['Player', '0-3 feet PPS', '0-3 Freq', '0-3 FGA per 75', '3-10 feet PPS', '3-10 Freq', '3-10 FGA per 75', '10-16 feet PPS', '10-16 Freq', '10-16 FGA per 75', '16-3P feet PPS', '16-3P Freq', '16-3P FGA per 75', '3P PPS', '3P Freq', '3P FGA per 75', 'PP75', 'TS+', 'MP'])

    for a, b in url_df.itertuples(index=False):
      player = a
      url = b
      url = "https://www.basketball-reference.com" + url


      wd.get(url)
      html = wd.page_source
      soup = BeautifulSoup(html)




      # scrape Playoff shooting distribution data
      for first_div in soup.find_all('div', attrs={'id': 'all_shooting-playoffs_shooting'}):
        second_div = first_div.find('div', attrs={'id': 'switcher_shooting-playoffs_shooting'})
        first_table = second_div.find('table', attrs={'id': 'playoffs_shooting'})
        foot = first_table.find('tfoot')

      rows = foot.find('tr')
      player_shooting_distribution_stats = [[td.getText() for td in rows.findAll('td')]]

      # remove empty rows
      player_shooting_distribution_stats = [e for e in player_shooting_distribution_stats if e != []]

      headers = ['dum1', 'dum2', 'Lg', 'dum3', 'G', 'MP', 'FG%', 'Dist.', 'dum4',
                '2Pfreq', '0-3freq', '3-10freq', '10-16freq', '16-3Pfreq', '3Pfreq',
                 'dum5', '2P', '0-3', '3-10', '10-16', '16-3P', '3P', 'dum6', '2Pass',
                 '3Pass', 'dum7', '%FGAdunks',	'#dunks',	'dum8', '%3PAcorner', '3P%corner', 'dum9', 'Heave Att.', 'Heave Mad.']

      shooting_dist_data = pd.DataFrame(player_shooting_distribution_stats, columns = headers)

      # drop empty cells
      shooting_dist_data = shooting_dist_data.drop(['dum1', 'dum2', 'dum3', 'dum4', 'dum5', 'dum6', 'dum7', 'dum8', 'dum9'], axis=1)

      tmp_df = pd.DataFrame(columns = ['Player', '0-3 feet PPS', '0-3 Freq', '0-3 FGA per 75', '3-10 feet PPS', '3-10 Freq', '3-10 FGA per 75', '10-16 feet PPS', '10-16 Freq', '10-16 FGA per 75', '16-3P feet PPS', '16-3P Freq', '16-3P FGA per 75', '3P PPS', '3P Freq', '3P FGA per 75', 'PP75', 'TS+', 'MP'])

      if int(shooting_dist_data['MP']) <= 1000:
        continue

      freq = [0, 0, 0, 0, 0]
      pps = [0, 0, 0, 0, 0]
      no_long2 = 0
      no_short2 = 0
      no_3 = 0
      freq_0 = float(shooting_dist_data['0-3freq'][0])
      freq_1 = float(shooting_dist_data['3-10freq'][0])
      if shooting_dist_data['10-16freq'][0] == '.000':
        no_short2 = 1
      freq_2 = float(shooting_dist_data['10-16freq'][0])
      if shooting_dist_data['16-3Pfreq'][0] == '.000':
        no_long2 = 1
      freq_3 = float(shooting_dist_data['16-3Pfreq'][0])
      if shooting_dist_data['3Pfreq'][0] == '.000':
        no_3 = 1
      freq_4 = float(shooting_dist_data['3Pfreq'][0])
      round(freq_0, 3)
      round(freq_1, 3)
      round(freq_2, 3)
      round(freq_3, 3)
      round(freq_4, 3)

      pps_0 = float(shooting_dist_data['0-3'])*2
      pps_1 = float(shooting_dist_data['3-10'])*2
      if no_short2 == 1:
        pps_2 = 0.00
      else:
        pps_2 = float(shooting_dist_data['10-16'])*2
      if no_long2 == 1:
        pps_3 = 0.00
      else:
        pps_3 = float(shooting_dist_data['16-3P'])*2
      if no_3 == 1:
        pps_4 = 0.00
      else:
        pps_4 = float(shooting_dist_data['3P'])*3
      round(pps_0, 3)
      round(pps_1, 3)
      round(pps_2, 3)
      round(pps_3, 3)
      round(pps_4, 3)





      # scrape Playoff FGA per 100 possessions
      for first_div in soup.find_all('div', attrs={'id': 'all_per_poss-playoffs_per_poss'}):
        second_div = first_div.find('div', attrs={'id': 'switcher_per_poss-playoffs_per_poss'})
        first_table = second_div.find('table', attrs={'id': 'playoffs_per_poss'})
        foot = first_table.find('tfoot')

      rows = foot.find('tr')
      player_per_100_data = [[td.getText() for td in rows.findAll('td')]]

      # remove empty rows
      player_per_100_data = [e for e in player_per_100_data if e != []]

      headers = ['Age', 'Tm', 'Lg', 'Pos', 'G', 'GS', 'MP', 'FG', 'FGA',
                 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%','FT', 'FTA', 'FT%',
                 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'dum1',
                 'ORtg', 'DRtg']

      playoff_fga = pd.DataFrame(player_per_100_data, columns = headers)

      # append dataframe
      playoff_fga = playoff_fga.drop(['Age', 'Tm', 'Lg', 'Pos', 'G', 'GS', 'MP', 'FG',
                 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%','FT', 'FTA', 'FT%',
                 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'dum1',
                 'ORtg', 'DRtg'], axis=1)
      playoff_fga_per_75_0 = (float(playoff_fga['FGA']) * .75) * freq_0
      playoff_fga_per_75_1 = (float(playoff_fga['FGA']) * .75) * freq_1
      playoff_fga_per_75_2 = (float(playoff_fga['FGA']) * .75) * freq_2
      playoff_fga_per_75_3 = (float(playoff_fga['FGA']) * .75) * freq_3
      playoff_fga_per_75_4 = (float(playoff_fga['FGA']) * .75) * freq_4
      playoff_fga_per_75_0 = '%.2f' % round(playoff_fga_per_75_0, 2)
      playoff_fga_per_75_1 = '%.2f' % round(playoff_fga_per_75_1, 2)
      playoff_fga_per_75_2 = '%.2f' % round(playoff_fga_per_75_2, 2)
      playoff_fga_per_75_3 = '%.2f' % round(playoff_fga_per_75_3, 2)
      playoff_fga_per_75_4 = '%.2f' % round(playoff_fga_per_75_4, 2)


      (pp75, ts, mp) = playoff_career_scoring_output(player, era_opponent_adj_playoff_per_75_df)
      tmp_df.loc[len(tmp_df)] = [player, pps_0, freq_0, playoff_fga_per_75_0, pps_1, freq_1, playoff_fga_per_75_1, pps_2, freq_2, playoff_fga_per_75_2, pps_3, freq_3, playoff_fga_per_75_3,pps_4, freq_4, playoff_fga_per_75_4, pp75, ts, mp]
      new_df = new_df.append(tmp_df)
      print(tmp_df)
      outfile = f"NBA_Playoff_all_shooting_data_df.csv"
      new_df.to_csv(outfile, index=False)
def playoff_career_scoring_output(player, postseason_data):
  player_data = postseason_data[(postseason_data.Player == player)]
  player_data = player_data[(player_data.Year >= 1997)]

  total_mp = 0
  total_pts = 0
  total_ts = 0
  mp_list = []

  # find total minutes a
  for row in player_data['MP']:
    mp_list.append(row)
    total_mp += row
  if total_mp == 0:
    total_mp = 1

  # find total PTS a
  i = 0
  for row in player_data['PTS']:
    total_pts += row * (mp_list[i] / total_mp)
    i = i + 1

  # find total TS+ a
  i = 0
  for row in player_data['TS+']:
    total_ts += row * (mp_list[i] / total_mp)
    i = i + 1

  round(total_pts, 2)
  total_pts = '%.2f' % round(total_pts, 2)
  total_ts = '%.2f' % round(total_ts, 2)
  return (total_pts, total_ts, total_mp)

In [4]:
#@title nba playoff ALL shooting zone scrapes (Playoffs-By-Playoffs)

def nba_all_shooting_data_pbp(url_df):

    era_opponent_adj_playoff_per_75_df = pd.read_csv('/content/era_opponent_adjusted_per75_data.csv')

    new_df = pd.DataFrame(columns = ['Playoffs', 'Player', '0-3 feet PPS', '0-3 Freq', '0-3 FGA per 75', '3-10 feet PPS', '3-10 Freq', '3-10 FGA per 75', '10-16 feet PPS', '10-16 Freq', '10-16 FGA per 75', '16-3P feet PPS', '16-3P Freq', '16-3P FGA per 75', '3P PPS', '3P Freq', '3P FGA per 75', 'FTA per 75', 'FT PPS', 'PP75', 'TS+', 'MP'])

    for a, b in url_df.itertuples(index=False):
      player = a
      url = b
      url = "https://www.basketball-reference.com" + url


      wd.get(url)
      html = wd.page_source
      soup = BeautifulSoup(html)




      # scrape Playoff shooting distribution data
      for first_div in soup.find_all('div', attrs={'id': 'all_shooting-playoffs_shooting'}):
        second_div = first_div.find('div', attrs={'id': 'switcher_shooting-playoffs_shooting'})
        first_table = second_div.find('table', attrs={'id': 'playoffs_shooting'})
        body = first_table.find('tbody')

      rows = body.findAll('tr')[0:]
      series_stats = [[td.getText() for td in rows[i].findAll(['td','th'])] for i in range(len(rows))]

      # remove empty rows
      series_stats = [e for e in series_stats if e != []]
      series_stats = ([i for i in series_stats if i[0] != 'Year'])

      headers = ['Season', 'dum1', 'dum2', 'Lg', 'dum3', 'G', 'MP', 'FG%', 'Dist.', 'dum4',
                '2Pfreq', '0-3freq', '3-10freq', '10-16freq', '16-3Pfreq', '3Pfreq',
                 'dum5', '2P', '0-3', '3-10', '10-16', '16-3P', '3P', 'dum6', '2Pass',
                 '3Pass', 'dum7', '%FGAdunks',	'#dunks',	'dum8', '%3PAcorner', '3P%corner', 'dum9', 'Heave Att.', 'Heave Mad.']

      shooting_dist_data = pd.DataFrame(series_stats, columns = headers)

      # drop empty cells
      shooting_dist_data = shooting_dist_data.drop(['dum1', 'dum2', 'dum3', 'dum4', 'dum5', 'dum6', 'dum7', 'dum8', 'dum9'], axis=1)

      # scrape Playoff FGA per 100 possessions
      for first_div in soup.find_all('div', attrs={'id': 'all_per_poss-playoffs_per_poss'}):
        second_div = first_div.find('div', attrs={'id': 'switcher_per_poss-playoffs_per_poss'})
        first_table = second_div.find('table', attrs={'id': 'playoffs_per_poss'})
        header = first_table.find('thead')
        body = first_table.find('tbody')

      rows = body.findAll('tr')[0:]
      player_per_100 = [[td.getText() for td in rows[i].findAll(['td','th'])] for i in range(len(rows))]

      # remove empty rows
      player_per_100 = [e for e in player_per_100 if e != []]
      player_per_100 = ([i for i in player_per_100 if i[0] != 'Year'])

      headers = [th.getText() for th in header.findAll('tr', limit=2)[0].findAll('th')]
      total_playoff_fga = pd.DataFrame(player_per_100, columns = headers)

      for idx, row in shooting_dist_data.iterrows():

        season = row['Season']

        tmp_df = pd.DataFrame(columns = ['Playoffs', 'Player', '0-3 feet PPS', '0-3 Freq', '0-3 FGA per 75', '3-10 feet PPS', '3-10 Freq', '3-10 FGA per 75', '10-16 feet PPS', '10-16 Freq', '10-16 FGA per 75', '16-3P feet PPS', '16-3P Freq', '16-3P FGA per 75', '3P PPS', '3P Freq', '3P FGA per 75', 'FTA per 75', 'FT PPS', 'PP75', 'TS+', 'MP'])

        freq = [0, 0, 0, 0, 0]
        pps = [0, 0, 0, 0, 0]
        no_0_3 = 0
        no_3_10 = 0
        no_10_16 = 0
        no_16_3P = 0
        no_3 = 0

        # Frequency from each zone

        if row['0-3freq'] == '.000':
          no_0_3 = 1
        try:
          freq_0_3 = float(row['0-3freq'])
        except:
          # no text at all for '0-3freq means that player took zero FGA this playoffs; skip
          continue
        if row['3-10freq'] == '.000':
          no_3_10 = 1
        freq_3_10 = float(row['3-10freq'])
        if row['10-16freq'] == '.000':
          no_10_16 = 1
        freq_10_16 = float(row['10-16freq'])
        if row['16-3Pfreq'] == '.000':
          no_16_3P = 1
        freq_16_3P = float(row['16-3Pfreq'])
        if row['3Pfreq'] == '.000':
          no_3 = 1
        freq_3 = float(row['3Pfreq'])




        round(freq_0_3, 3)
        round(freq_3_10, 3)
        round(freq_10_16, 3)
        round(freq_16_3P, 3)
        round(freq_3, 3)

        # PPS from each zone

        if no_0_3 == 1:
          pps_0 = 0.00
        else:
          pps_0 = float(row['0-3'])*2
        if no_3_10 == 1:
          pps_1 = 0.00
        else:
          pps_1 = float(row['3-10'])*2
        if no_10_16 == 1:
          pps_2 = 0.00
        else:
          pps_2 = float(row['10-16'])*2
        if no_16_3P == 1:
          pps_3 = 0.00
        else:
          pps_3 = float(row['16-3P'])*2
        if no_3 == 1:
          pps_4 = 0.00
        else:
          pps_4 = float(row['3P'])*3
        round(pps_0, 3)
        round(pps_1, 3)
        round(pps_2, 3)
        round(pps_3, 3)
        round(pps_4, 3)

        # FGA per 75 from each zone

        playoff_fga = total_playoff_fga[(total_playoff_fga['Season'] == row['Season'])]

        playoff_fga_per_75_0 = (float(playoff_fga['FGA']) * .75) * freq_0_3
        playoff_fga_per_75_1 = (float(playoff_fga['FGA']) * .75) * freq_3_10
        playoff_fga_per_75_2 = (float(playoff_fga['FGA']) * .75) * freq_10_16
        playoff_fga_per_75_3 = (float(playoff_fga['FGA']) * .75) * freq_16_3P
        playoff_fga_per_75_4 = (float(playoff_fga['FGA']) * .75) * freq_3
        playoff_fga_per_75_0 = '%.2f' % round(playoff_fga_per_75_0, 2)
        playoff_fga_per_75_1 = '%.2f' % round(playoff_fga_per_75_1, 2)
        playoff_fga_per_75_2 = '%.2f' % round(playoff_fga_per_75_2, 2)
        playoff_fga_per_75_3 = '%.2f' % round(playoff_fga_per_75_3, 2)
        playoff_fga_per_75_4 = '%.2f' % round(playoff_fga_per_75_4, 2)


        # FTA per 75
        no_fta = 0
        playoff_fta_per_75_ = (float(playoff_fga['FTA']) * .75)
        if str(playoff_fga['FTA'].item()) == '0.0':
          no_fta = 1
        if no_fta == 1:
          playoff_ft_pps = 0
        else:
          playoff_ft_pps = (float(playoff_fga['FT%']) * 2)

        sub_2 = season[5:7]
        if (sub_2 == '00'):
          single_year_format = 2000
        else:
          single_year_format = season[0:2]
          single_year_format = single_year_format + sub_2
          single_year_format = single_year_format.strip()
          single_year_format = int(single_year_format)

        era_opponent_adj_playoff_per_75_df['Player'] = era_opponent_adj_playoff_per_75_df['Player'].astype(str)
        player_season = era_opponent_adj_playoff_per_75_df[(era_opponent_adj_playoff_per_75_df['Player'] == player) & (era_opponent_adj_playoff_per_75_df['Year'] == single_year_format)]
        pp75 = float(player_season['PTS'])
        ts = float(player_season['TS+'])
        mp = int(player_season['MP'])

        tmp_df.loc[len(tmp_df)] = [single_year_format, player, pps_0, freq_0_3, playoff_fga_per_75_0, pps_1, freq_3_10, playoff_fga_per_75_1, pps_2, freq_10_16, playoff_fga_per_75_2, pps_3, freq_16_3P, playoff_fga_per_75_3,pps_4, freq_3, playoff_fga_per_75_4, playoff_fta_per_75_, playoff_ft_pps, pp75, ts, mp]
        print(tmp_df)
        new_df = pd.concat([new_df, tmp_df], ignore_index=False)
        new_df = new_df.reset_index(drop=True)
      outfile = f"NBA_Playoff_all_shooting_data_season_by_season_df.csv"
      new_df.to_csv(outfile, index=False)

In [ ]:
player_urls = pd.read_csv('/content/tmp.csv', index_col=False, encoding='utf8')
player_urls = player_urls.drop(columns=['MP'])
nba_all_shooting_data_pbp(player_urls)

In [20]:
#@title X Year Peaks Shooting Function

# returns if all values in a column are unique.
# used to determine if a player played for same team across x year stretch
def is_unique(s):
    a = s.to_numpy()
    return (a[0] == a).all()


# def x_yearpeak(source_df, number_of_seasons_peak):
def x_yearpeak(source_df, number_of_seasons_peak):
  x_year_peak = pd.DataFrame(columns = ['Years', 'Player', '0-3 feet PPS', '0-3 Freq', '0-3 FGA per 75', '3-10 feet PPS', '3-10 Freq', '3-10 FGA per 75', '10-16 feet PPS', '10-16 Freq', '10-16 FGA per 75', '16-3P feet PPS', '16-3P Freq', '16-3P FGA per 75', '3P PPS', '3P Freq', '3P FGA per 75', 'FTA per 75', 'FT PPS', 'PP75', 'TS+', 'MP'])

  player_finished = []

  for idx, player in source_df.iterrows():

    # if player has not already been through function call
    if player['Player'] not in player_finished:
      single_player_df = source_df[(source_df['Player'] == player['Player'])]
      single_player_df = single_player_df.reset_index(drop=True)
      number_of_total_player_seasons = len(single_player_df.index)

      try:
        for season_index in range(0, number_of_total_player_seasons - (number_of_seasons_peak - 1)):

          years = ''
          non_consecutive_years = 0

          stretch_of_peak_seasons = single_player_df.iloc[season_index: season_index+number_of_seasons_peak].copy()
          mp = int(stretch_of_peak_seasons['MP'].sum())

          stretch_of_peak_seasons[f'zero_3_PPS_Portion'] = stretch_of_peak_seasons[f'0-3 feet PPS'] * (stretch_of_peak_seasons['MP'] / mp)
          stretch_of_peak_seasons[f'zero_3_freq_Portion'] = stretch_of_peak_seasons[f'0-3 Freq'] * (stretch_of_peak_seasons['MP'] / mp)
          stretch_of_peak_seasons[f'zero_3_fga_Portion'] = stretch_of_peak_seasons[f'0-3 FGA per 75'] * (stretch_of_peak_seasons['MP'] / mp)

          stretch_of_peak_seasons[f'3_10_PPS_Portion'] = stretch_of_peak_seasons[f'3-10 feet PPS'] * (stretch_of_peak_seasons['MP'] / mp)
          stretch_of_peak_seasons[f'3_10_freq_Portion'] = stretch_of_peak_seasons[f'3-10 Freq'] * (stretch_of_peak_seasons['MP'] / mp)
          stretch_of_peak_seasons[f'3_10_fga_Portion'] = stretch_of_peak_seasons[f'3-10 FGA per 75'] * (stretch_of_peak_seasons['MP'] / mp)

          stretch_of_peak_seasons[f'10_16_PPS_Portion'] = stretch_of_peak_seasons[f'10-16 feet PPS'] * (stretch_of_peak_seasons['MP'] / mp)
          stretch_of_peak_seasons[f'10_16_freq_Portion'] = stretch_of_peak_seasons[f'10-16 Freq'] * (stretch_of_peak_seasons['MP'] / mp)
          stretch_of_peak_seasons[f'10_16_fga_Portion'] = stretch_of_peak_seasons[f'10-16 FGA per 75'] * (stretch_of_peak_seasons['MP'] / mp)

          stretch_of_peak_seasons[f'16_3P_PPS_Portion'] = stretch_of_peak_seasons[f'16-3P feet PPS'] * (stretch_of_peak_seasons['MP'] / mp)
          stretch_of_peak_seasons[f'16_3P_freq_Portion'] = stretch_of_peak_seasons[f'16-3P Freq'] * (stretch_of_peak_seasons['MP'] / mp)
          stretch_of_peak_seasons[f'16_3P_fga_Portion'] = stretch_of_peak_seasons[f'16-3P FGA per 75'] * (stretch_of_peak_seasons['MP'] / mp)

          stretch_of_peak_seasons[f'3P_PPS_Portion'] = stretch_of_peak_seasons[f'3P PPS'] * (stretch_of_peak_seasons['MP'] / mp)
          stretch_of_peak_seasons[f'3P_freq_Portion'] = stretch_of_peak_seasons[f'3P Freq'] * (stretch_of_peak_seasons['MP'] / mp)
          stretch_of_peak_seasons[f'3P_fga_Portion'] = stretch_of_peak_seasons[f'3P FGA per 75'] * (stretch_of_peak_seasons['MP'] / mp)

          stretch_of_peak_seasons[f'pp75_Portion'] = stretch_of_peak_seasons[f'PP75'] * (stretch_of_peak_seasons['MP'] / mp)
          stretch_of_peak_seasons[f'ts_Portion'] = stretch_of_peak_seasons[f'TS+'] * (stretch_of_peak_seasons['MP'] / mp)

          stretch_of_peak_seasons[f'fta_Portion'] = stretch_of_peak_seasons[f'FTA per 75'] * (stretch_of_peak_seasons['MP'] / mp)
          stretch_of_peak_seasons[f'ft_pps_Portion'] = stretch_of_peak_seasons[f'FT PPS'] * (stretch_of_peak_seasons['MP'] / mp)

          zero_3_PPS = stretch_of_peak_seasons[f'zero_3_PPS_Portion'].sum().round(2)
          zero_3_freq = stretch_of_peak_seasons[f'zero_3_freq_Portion'].sum().round(2)
          zero_3_fga = stretch_of_peak_seasons[f'zero_3_fga_Portion'].sum().round(2)
          _3_10_PPS = stretch_of_peak_seasons[f'3_10_PPS_Portion'].sum().round(2)
          _3_10_freq = stretch_of_peak_seasons[f'3_10_freq_Portion'].sum().round(2)
          _3_10_fga = stretch_of_peak_seasons[f'3_10_fga_Portion'].sum().round(2)
          _10_16_PPS = stretch_of_peak_seasons[f'10_16_PPS_Portion'].sum().round(2)
          _10_16_freq = stretch_of_peak_seasons[f'10_16_freq_Portion'].sum().round(2)
          _10_16_fga = stretch_of_peak_seasons[f'10_16_fga_Portion'].sum().round(2)
          _16_3P_PPS = stretch_of_peak_seasons[f'16_3P_PPS_Portion'].sum().round(2)
          _16_3P_freq = stretch_of_peak_seasons[f'16_3P_freq_Portion'].sum().round(2)
          _16_3P_fga = stretch_of_peak_seasons[f'16_3P_fga_Portion'].sum().round(2)
          _3_PPS = stretch_of_peak_seasons[f'3P_PPS_Portion'].sum().round(2)
          _3_freq = stretch_of_peak_seasons[f'3P_freq_Portion'].sum().round(2)
          _3_fga = stretch_of_peak_seasons[f'3P_fga_Portion'].sum().round(2)
          pp75 = stretch_of_peak_seasons[f'pp75_Portion'].sum().round(2)
          ts = stretch_of_peak_seasons[f'ts_Portion'].sum().round(2)
          fta = stretch_of_peak_seasons[f'fta_Portion'].sum().round(2)
          ft_pps = stretch_of_peak_seasons[f'ft_pps_Portion'].sum().round(2)


          prev_year = stretch_of_peak_seasons.iloc[0]['Playoffs']
          for year in stretch_of_peak_seasons['Playoffs']:
            if year - prev_year != 1 and year - prev_year != 0:
              stretch_of_peak_seasons["Playoffs"] = stretch_of_peak_seasons["Playoffs"].astype(str)
              years = ', '.join(stretch_of_peak_seasons["Playoffs"])
              non_consecutive_years = 1
              break
            prev_year = year
          if non_consecutive_years == 0:
            years = f"{stretch_of_peak_seasons.iloc[0]['Playoffs']} - {stretch_of_peak_seasons.iloc[number_of_seasons_peak-1]['Playoffs']}"

          cols = ['Years', 'Player', '0-3 feet PPS', '0-3 Freq', '0-3 FGA per 75', '3-10 feet PPS', '3-10 Freq', '3-10 FGA per 75', '10-16 feet PPS', '10-16 Freq', '10-16 FGA per 75', '16-3P feet PPS', '16-3P Freq', '16-3P FGA per 75', '3P PPS', '3P Freq', '3P FGA per 75', 'FTA per 75', 'FT PPS', 'PP75', 'TS+', 'MP']
          df_temp = pd.DataFrame([[years, stretch_of_peak_seasons.iloc[0]['Player'], zero_3_PPS, zero_3_freq, zero_3_fga, _3_10_PPS, _3_10_freq, _3_10_fga, _10_16_PPS, _10_16_freq, _10_16_fga, _16_3P_PPS, _16_3P_freq, _16_3P_fga, _3_PPS, _3_freq, _3_fga, fta, ft_pps, pp75, ts, mp]], columns=cols)
          x_year_peak = pd.concat([x_year_peak, df_temp], ignore_index=False)
          outfile = f"{number_of_seasons_peak}_year_peak_data.csv"
          x_year_peak.to_csv(outfile, index=False)
        player_finished.append(stretch_of_peak_seasons.iloc[0]['Player'])
      except:
        continue

In [ ]:
#@title Run X Year Peaks Shooting Function
# 2
years = 2

shooting_data = pd.read_csv('/content/NBA_Playoff_all_shooting_data_season_by_season_df.csv', index_col=False, encoding='utf8')
x_yearpeak(shooting_data, years)
import_player_PTS_peaks_df = pd.read_csv(f'{years}_year_peak_data.csv', encoding='utf8')
outfile = f"{years}_year_peak_data.csv"
import_player_PTS_peaks_df.to_csv(outfile, index=False)
print(import_player_PTS_peaks_df)

# 2
years = 3

shooting_data = pd.read_csv('/content/NBA_Playoff_all_shooting_data_season_by_season_df.csv', index_col=False, encoding='utf8')
x_yearpeak(shooting_data, years)
import_player_PTS_peaks_df = pd.read_csv(f'{years}_year_peak_data.csv', encoding='utf8')
outfile = f"{years}_year_peak_data.csv"
import_player_PTS_peaks_df.to_csv(outfile, index=False)
print(import_player_PTS_peaks_df)

# 2
years = 4

shooting_data = pd.read_csv('/content/NBA_Playoff_all_shooting_data_season_by_season_df.csv', index_col=False, encoding='utf8')
x_yearpeak(shooting_data, years)
import_player_PTS_peaks_df = pd.read_csv(f'{years}_year_peak_data.csv', encoding='utf8')
outfile = f"{years}_year_peak_data.csv"
import_player_PTS_peaks_df.to_csv(outfile, index=False)
print(import_player_PTS_peaks_df)

years = 5

shooting_data = pd.read_csv('/content/NBA_Playoff_all_shooting_data_season_by_season_df.csv', index_col=False, encoding='utf8')
x_yearpeak(shooting_data, years)
import_player_PTS_peaks_df = pd.read_csv(f'{years}_year_peak_data.csv', encoding='utf8')
outfile = f"{years}_year_peak_data.csv"
import_player_PTS_peaks_df.to_csv(outfile, index=False)
print(import_player_PTS_peaks_df)

years = 6

shooting_data = pd.read_csv('/content/NBA_Playoff_all_shooting_data_season_by_season_df.csv', index_col=False, encoding='utf8')
x_yearpeak(shooting_data, years)
import_player_PTS_peaks_df = pd.read_csv(f'{years}_year_peak_data.csv', encoding='utf8')
outfile = f"{years}_year_peak_data.csv"
import_player_PTS_peaks_df.to_csv(outfile, index=False)
print(import_player_PTS_peaks_df)

years = 7

shooting_data = pd.read_csv('/content/NBA_Playoff_all_shooting_data_season_by_season_df.csv', index_col=False, encoding='utf8')
x_yearpeak(shooting_data, years)
import_player_PTS_peaks_df = pd.read_csv(f'{years}_year_peak_data.csv', encoding='utf8')
outfile = f"{years}_year_peak_data.csv"
import_player_PTS_peaks_df.to_csv(outfile, index=False)
print(import_player_PTS_peaks_df)

years = 8

shooting_data = pd.read_csv('/content/NBA_Playoff_all_shooting_data_season_by_season_df.csv', index_col=False, encoding='utf8')
x_yearpeak(shooting_data, years)
import_player_PTS_peaks_df = pd.read_csv(f'{years}_year_peak_data.csv', encoding='utf8')
outfile = f"{years}_year_peak_data.csv"
import_player_PTS_peaks_df.to_csv(outfile, index=False)
print(import_player_PTS_peaks_df)

years = 9

shooting_data = pd.read_csv('/content/NBA_Playoff_all_shooting_data_season_by_season_df.csv', index_col=False, encoding='utf8')
x_yearpeak(shooting_data, years)
import_player_PTS_peaks_df = pd.read_csv(f'{years}_year_peak_data.csv', encoding='utf8')
outfile = f"{years}_year_peak_data.csv"
import_player_PTS_peaks_df.to_csv(outfile, index=False)
print(import_player_PTS_peaks_df)

years = 10

shooting_data = pd.read_csv('/content/NBA_Playoff_all_shooting_data_season_by_season_df.csv', index_col=False, encoding='utf8')
x_yearpeak(shooting_data, years)
import_player_PTS_peaks_df = pd.read_csv(f'{years}_year_peak_data.csv', encoding='utf8')
outfile = f"{years}_year_peak_data.csv"
import_player_PTS_peaks_df.to_csv(outfile, index=False)
print(import_player_PTS_peaks_df)

In [28]:
#@title Import X Year Peaks Shooting Data
# 2
all_peaks = pd.DataFrame()

import_adjpts_twopeaks_df = pd.read_csv('2_year_peak_data.csv', encoding='utf8')
all_peaks = pd.concat([all_peaks, import_adjpts_twopeaks_df], ignore_index=False)

import_adjpts_threepeaks_df = pd.read_csv('3_year_peak_data.csv', encoding='utf8')
all_peaks = pd.concat([all_peaks, import_adjpts_threepeaks_df], ignore_index=False)

import_adjpts_4peaks_df = pd.read_csv('4_year_peak_data.csv', encoding='utf8')
all_peaks = pd.concat([all_peaks, import_adjpts_4peaks_df], ignore_index=False)

import_adjpts_fivepeaks_df = pd.read_csv('5_year_peak_data.csv', encoding='utf8')
all_peaks = pd.concat([all_peaks, import_adjpts_fivepeaks_df], ignore_index=False)

import_adjpts_6peaks_df = pd.read_csv('6_year_peak_data.csv', encoding='utf8')
all_peaks = pd.concat([all_peaks, import_adjpts_6peaks_df], ignore_index=False)

import_adjpts_7peaks_df = pd.read_csv('7_year_peak_data.csv', encoding='utf8')
all_peaks = pd.concat([all_peaks, import_adjpts_7peaks_df], ignore_index=False)

import_adjpts_eightpeaks_df = pd.read_csv('8_year_peak_data.csv', encoding='utf8')
all_peaks = pd.concat([all_peaks, import_adjpts_eightpeaks_df], ignore_index=False)

import_adjpts_9peaks_df = pd.read_csv('9_year_peak_data.csv', encoding='utf8')
all_peaks = pd.concat([all_peaks, import_adjpts_9peaks_df], ignore_index=False)

import_adjpts_10peaks_df = pd.read_csv('10_year_peak_data.csv', encoding='utf8')
all_peaks = pd.concat([all_peaks, import_adjpts_10peaks_df], ignore_index=False)

all_peaks = all_peaks.sort_values('MP', ascending=False)
all_peaks = all_peaks.to_csv("All_Shooting_Data_Peaks.csv", index=False)

In [ ]:
#@title Add Approx_Scoring_Value
all_peaks= pd.read_csv('/content/All_Shooting_Data_Peaks.csv', encoding='utf8')
graph_data = pd.read_csv('/content/series_data_52_22.csv', encoding='utf8', index_col=False)
graph_data = graph_data[(graph_data['PTS/G'] >= 10)]
graph_data = graph_data[(graph_data['MP'] >= 100)]


for idx, row in all_peaks.iterrows():
  pts = row['PP75']
  ts = row['TS+']
  mp = row['MP']
  pts_percentile = stats.percentileofscore(graph_data['PTS/G'], pts)
  ts_percentile = stats.percentileofscore(graph_data['TS+'], ts)
  all_peaks.loc[idx, 'Approx_Scoring_Val'] =  (0.5 * pts_percentile) + (0.5 * ts_percentile)
print(all_peaks)

In [30]:
#@title Prepare Plotting Data
plotting = all_peaks[(all_peaks['MP'] >= 1000)]

plotting.to_csv('All_Shooting_Data_Peaks_1000_MP.csv', index=False)
plotting = plotting[(plotting['PP75'] >= 15)]
plotting.to_csv('All_Shooting_Data_Peaks_1000_MP_15_PP75.csv', index=False)

In [59]:
#@title Prepare 2P Plotting Data
plotting = all_peaks[(all_peaks['MP'] >= 1000)]

plotting.to_csv('All_Shooting_Data_Peaks_1000_MP.csv', index=False)
plotting = plotting[(plotting['PP75'] >= 15)]
plotting.to_csv('All_Shooting_Data_Peaks_1000_MP_15_PP75.csv', index=False)

graph_data = plotting.copy()
graph_data['2P Jumpshot FGA per 75'] = graph_data['3-10 FGA per 75'] + graph_data['10-16 FGA per 75'] + graph_data['16-3P FGA per 75']
total_freq = graph_data['3-10 Freq'] + graph_data['10-16 Freq'] + graph_data['16-3P Freq']
graph_data['2P Jumpshot PPS'] = (((graph_data['3-10 Freq']) / total_freq) * graph_data['3-10 feet PPS']) + (((graph_data['10-16 Freq']) / total_freq) * graph_data['10-16 feet PPS']) + (((graph_data['16-3P Freq']) / total_freq) * graph_data['16-3P feet PPS'])

graph_data = graph_data[(graph_data['2P Jumpshot FGA per 75'] >= 5)]

graph_data.to_csv('All_Shooting_Data_Peaks_1000_MP_15_PP75_5_2P_Jumpshots.csv', index=False)

In [110]:
#@title Prepare 3-16 Feet Plotting Data
plotting = all_peaks[(all_peaks['MP'] >= 1000)]

plotting.to_csv('All_Shooting_Data_Peaks_1000_MP.csv', index=False)
plotting = plotting[(plotting['PP75'] >= 15)]
plotting.to_csv('All_Shooting_Data_Peaks_1000_MP_15_PP75.csv', index=False)

graph_data = plotting.copy()
graph_data['3-16 FGA per 75'] = graph_data['3-10 FGA per 75'] + graph_data['10-16 FGA per 75']
total_freq = graph_data['3-10 Freq'] + graph_data['10-16 Freq']
graph_data['3-16 feet PPS'] = (((graph_data['3-10 Freq']) / total_freq) * graph_data['3-10 feet PPS']) + (((graph_data['10-16 Freq']) / total_freq) * graph_data['10-16 feet PPS'])
graph_data = graph_data[(graph_data['3-16 FGA per 75'] >= 5)]

graph_data.to_csv('All_Shooting_Data_Peaks_1000_MP_15_PP75_5_3-16_Jumpshots.csv', index=False)

In [126]:
#@title Prepare > 3 Feet FGA Plotting Data
plotting = all_peaks[(all_peaks['MP'] >= 1000)]

plotting.to_csv('All_Shooting_Data_Peaks_1000_MP.csv', index=False)
plotting = plotting[(plotting['PP75'] >= 15)]
plotting.to_csv('All_Shooting_Data_Peaks_1000_MP_15_PP75.csv', index=False)

graph_data = plotting.copy()
graph_data['>3 Feet FGA per 75'] = graph_data['3-10 FGA per 75'] + graph_data['10-16 FGA per 75'] + graph_data['16-3P FGA per 75'] + graph_data['3P FGA per 75']
total_freq = graph_data['3-10 Freq'] + graph_data['10-16 Freq'] + graph_data['16-3P Freq'] + graph_data['3P Freq']
graph_data['>3 Feet PPS'] = (((graph_data['3-10 Freq']) / total_freq) * graph_data['3-10 feet PPS']) + (((graph_data['10-16 Freq']) / total_freq) * graph_data['10-16 feet PPS']) + (((graph_data['16-3P Freq']) / total_freq) * graph_data['16-3P feet PPS']) + (((graph_data['3P Freq']) / total_freq) * graph_data['3P PPS'])
graph_data = graph_data[(graph_data['>3 Feet FGA per 75'] >= 10)]

graph_data.to_csv('All_Shooting_Data_Peaks_1000_MP_15_PP75_10_All_Jumpshots.csv', index=False)

In [13]:
#@title Output Multiple Players' Peak Stretches
def printMultiplePlayerStretches(series_stretches_file, mp_min, mp_max, list_length, unique_players, best_or_worst, first_or_last):

  series_data_df = pd.read_csv(series_stretches_file, encoding='utf8', index_col=False)
  series_data_df = series_data_df[(series_data_df['MP'] >= mp_min)]
  series_data_df = series_data_df[(series_data_df['MP'] <= mp_max)]

  if best_or_worst == 'best':
    series_data_df = series_data_df.sort_values('Approx_Scoring_Val', ascending=False)
  elif best_or_worst == 'worst':
    series_data_df = series_data_df.sort_values('Approx_Scoring_Val', ascending=True)
  series_data_df['Approx_Scoring_Val'] = series_data_df['Approx_Scoring_Val'].round(2)


  if unique_players == 1:
    if first_or_last == 'first':
      series_data_df = series_data_df.drop_duplicates('Player', keep='first')
    elif first_or_last == 'last':
      series_data_df = series_data_df.drop_duplicates('Player', keep='last')
  series_data_df = series_data_df.reset_index(drop=True)
  series_data_df['Historical Player Rank'] = series_data_df.index
  #print(series_data_df.shape[0])
  #series_data_df = series_data_df[(series_data_df['Player'] == 'James Harden') | (series_data_df['Player'] == 'Bruce Bowen') | (series_data_df['Historical Player Rank'] == 0) | (series_data_df['Historical Player Rank'] == 681)]
  #series_data_df = series_data_df[(series_data_df['Player'] == 'Manu Ginóbili') | (series_data_df['Player'] == 'Kyrie Irving') | (series_data_df['Player'] == 'Tim Duncan')]
  series_data_df = series_data_df[(series_data_df['Historical Player Rank'] <= list_length-1)]
  series_data_df = series_data_df.drop(columns='Historical Player Rank')
  #col_list = series_data_df.Player.values.tolist()
  #print(col_list)




  titlestring = f"\tBest Playoff Scoring Stretches (max 1 per player)\n\t\t\t\t Minimum {mp_min} MP\n"

  print(titlestring)

  print(series_data_df)

  series_data_df.to_csv(f'Top {mp_min} MP scoring stretches.csv',index=False)

  print("\n\t'Approx_Scoring_Val' = Historical percentile of opponent adjusted PTS/G (1952-2022) * 0.5 + \n\t\t\t\tHistorical percentile of TS+ (1952-2022)* 0.5\n\n\tPTS/G adjusted based on regular season rDefRtg of playoff opponents\n\tTS+ adjusted based on opponents' TS% allowed in regular season\n\t1952-73 pace & TS% allowed both estimated")

In [ ]:
printMultiplePlayerStretches('All_Shooting_Data_Peaks_1000_MP_15_PP75.csv', 3000, 11305, 200, 1, "best", "first")

In [ ]:
two = pd.read_csv('/content/Top 2000 MP scoring stretches.csv', encoding='utf8')
three = pd.read_csv('/content/Top 3000 MP scoring stretches.csv', encoding='utf8')

both = pd.concat([two, three], ignore_index=False)
both = both.sort_values('MP', ascending=False)
both = both.drop_duplicates('Player', keep='first')
print(both)

both.to_csv('Shooting_Peaks.csv', index=False)

In [ ]:
shaq = plotting[(plotting['Player'] == 'James Harden')]
shaq = shaq.sort_values('Approx_Scoring_Val', ascending=False)
shaq = shaq[(shaq['Approx_Scoring_Val'] >= 77)]
print(shaq)

In [ ]:
 'FTA per 75', 'FT PPS',

**PLOT SHOOTING DISTRIBUTION DATA PEAKS**

In [ ]:
#@title PP75 FTA Graph
graph_data =  pd.read_csv('Shooting_Peaks.csv', encoding='utf8')

Tony = (graph_data['Player']=='James Harden')
graph_data['color']= np.where(Tony==True , "r", "#00B2A9")

size = 100 * ((graph_data['MP'] - (graph_data['MP'].min() - 1)) / (graph_data['MP'].max()+2 - (graph_data['MP'].min() - 1)))
plt.figure(figsize=(20,12), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="PP75", y="FTA per 75", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='>3 Feet Playoff Scoring Peaks (1997-2022) > 15 PP75 >2000 MP', xlabel='PP75', ylabel='FTA per 75')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='James Harden', markerfacecolor='r',  markersize=10)]

ax.legend(handles=legend_elements, loc='lower right')

# ridge regression
x_pred = graph_data['PP75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['FTA per 75']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
ax.plot(x_pred, pred, color="k", linewidth=3)

graph_data = graph_data[(graph_data['Player'] != 'James Harden')]
label_point(graph_data['PP75'], graph_data['FTA per 75'], graph_data['Player'], plt.gca())
plt.savefig(f'PP75_FTA_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title FGA FTA Graph
graph_data =  pd.read_csv('Shooting_Peaks.csv', encoding='utf8')

graph_data['FGA'] = graph_data['0-3 FGA per 75'] + graph_data['3-10 FGA per 75'] + graph_data['10-16 FGA per 75'] + graph_data['16-3P FGA per 75'] + graph_data['3P FGA per 75']

Tony = (graph_data['Player']=='James Harden')
graph_data['color']= np.where(Tony==True , "r", "#00B2A9")

size = 100 * ((graph_data['MP'] - (graph_data['MP'].min() - 1)) / (graph_data['MP'].max()+2 - (graph_data['MP'].min() - 1)))
plt.figure(figsize=(20,12), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="FGA", y="FTA per 75", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='>3 Feet Playoff Scoring Peaks (1997-2022) > 15 PP75 >2000 MP', xlabel='FGA', ylabel='FTA per 75')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='James Harden', markerfacecolor='r',  markersize=10)]

ax.legend(handles=legend_elements, loc='lower right')

# ridge regression
x_pred = graph_data['FGA']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['FTA per 75']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
ax.plot(x_pred, pred, color="k", linewidth=3)

graph_data = graph_data[(graph_data['Player'] != 'James Harden')]
label_point(graph_data['FGA'], graph_data['FTA per 75'], graph_data['Player'], plt.gca())
plt.savefig(f'FGA_FTA_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title FGA FTA Graph
graph_data =  pd.read_csv('Shooting_Peaks.csv', encoding='utf8')

graph_data['FGA'] = graph_data['0-3 FGA per 75'] + graph_data['3-10 FGA per 75'] + graph_data['10-16 FGA per 75'] + graph_data['16-3P FGA per 75'] + graph_data['3P FGA per 75']

Tony = (graph_data['Player']=='James Harden')
graph_data['color']= np.where(Tony==True , "r", "#00B2A9")

size = 100 * ((graph_data['MP'] - (graph_data['MP'].min() - 1)) / (graph_data['MP'].max()+2 - (graph_data['MP'].min() - 1)))
plt.figure(figsize=(20,12), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="FTA per 75", y="FT PPS", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='>3 Feet Playoff Scoring Peaks (1997-2022) > 15 PP75 >2000 MP', xlabel='FTA per 75', ylabel='FT PPS')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='James Harden', markerfacecolor='r',  markersize=10)]

ax.legend(handles=legend_elements, loc='lower right')

# ridge regression
x_pred = graph_data['FTA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['FT PPS']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
ax.plot(x_pred, pred, color="k", linewidth=3)

graph_data = graph_data[(graph_data['Player'] != 'James Harden')]
label_point(graph_data['FTA per 75'], graph_data['FT PPS'], graph_data['Player'], plt.gca())
plt.savefig(f'FGA_FTA_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title All Jumpshots PPS Graph
graph_data =  pd.read_csv('/content/All_Jumpshots_Peaks.csv', encoding='utf8')

Tony = (graph_data['Player']=='Jrue Holiday')
graph_data['color']= np.where(Tony==True , "#EF426F", "#00B2A9")

size = 100 * ((graph_data['PP75'] - (graph_data['PP75'].min() - 1)) / (graph_data['PP75'].max()+2 - (graph_data['PP75'].min() - 1)))
plt.figure(figsize=(20,12), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x=">3 Feet FGA per 75", y=">3 Feet PPS", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='>3 Feet Playoff Scoring Peaks (1997-2022) > 15 PP75 >2000 MP', xlabel='>3 Feet FGA per 75', ylabel='>3 Feet PPS')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Jrue Holiday', markerfacecolor='#EF426F',  markersize=10)]

ax.legend(handles=legend_elements, loc='lower right')

# ridge regression
x_pred = graph_data['>3 Feet FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['>3 Feet PPS']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
ax.plot(x_pred, pred, color="k", linewidth=3)

graph_data = graph_data[(graph_data['Player'] != 'Jrue Holiday')]
label_point(graph_data['>3 Feet FGA per 75'], graph_data['>3 Feet PPS'], graph_data['Player'], plt.gca())
plt.savefig(f'>3 Feet_PPS_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title All Jumpshots TS+ Graph
graph_data =  pd.read_csv('/content/All_Jumpshots_Peaks.csv', encoding='utf8')

Tony = (graph_data['Player']=='Jrue Holiday')
graph_data['color']= np.where(Tony==True , "#EF426F", "#00B2A9")

size = 100 * ((graph_data['PP75'] - (graph_data['PP75'].min() - 1)) / (graph_data['PP75'].max()+2 - (graph_data['PP75'].min() - 1)))
plt.figure(figsize=(20,12), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x=">3 Feet FGA per 75", y="TS+", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='>3 Feet Playoff Scoring Peaks (1997-2022) > 15 PP75 >2000 MP', xlabel='>3 Feet FGA per 75', ylabel='TS+')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Jrue Holiday', markerfacecolor='#EF426F',  markersize=10)]

ax.legend(handles=legend_elements, loc='lower right')

# ridge regression
x_pred = graph_data['>3 Feet FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['TS+']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
ax.plot(x_pred, pred, color="k", linewidth=3)

graph_data = graph_data[(graph_data['Player'] != 'Jrue Holiday')]
label_point(graph_data['>3 Feet FGA per 75'], graph_data['TS+'], graph_data['Player'], plt.gca())
plt.savefig(f'>3 Feet_TS_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title 3-16 PPS Graph
graph_data =  pd.read_csv('/content/3-16_Feet_Jumpshots_Peaks.csv', encoding='utf8')

Tony = (graph_data['Player']=='Tim Duncan')
graph_data['color']= np.where(Tony==True , "#EF426F", "#00B2A9")
graph_data['color'] = np.where((graph_data.Player == 'Kobe Bryant'),'y', graph_data.color)

size = 100 * ((graph_data['PP75'] - (graph_data['PP75'].min() - 1)) / (graph_data['PP75'].max()+2 - (graph_data['PP75'].min() - 1)))
plt.figure(figsize=(20,12), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="3-16 FGA per 75", y="3-16 feet PPS", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='3-16 Feet Playoff Scoring Peaks (1997-2022) > 15 PP75 >2000 MP', xlabel='3-16 feet FGA per 75', ylabel='3-16 feet PPS')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Tim Duncan', markerfacecolor='#EF426F',  markersize=10),
                   Line2D([0], [0], marker='o', color='#EAEAF2', label='Kobe Bryant', markerfacecolor='y',  markersize=10)]
ax.legend(handles=legend_elements, loc='lower right')

# ridge regression
x_pred = graph_data['3-16 FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['3-16 feet PPS']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
ax.plot(x_pred, pred, color="k", linewidth=3)

graph_data = graph_data[(graph_data['Player'] != 'Tim Duncan')]
graph_data = graph_data[(graph_data['Player'] != 'Kobe Bryant')]
label_point(graph_data['3-16 FGA per 75'], graph_data['3-16 feet PPS'], graph_data['Player'], plt.gca())
plt.savefig(f'3-16_PPS_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title 3-16 TS+ Graph
graph_data =  pd.read_csv('/content/3-16_Feet_Jumpshots_Peaks.csv', encoding='utf8')

Tony = (graph_data['Player']=='Tim Duncan')
graph_data['color']= np.where(Tony==True , "#EF426F", "#00B2A9")
graph_data['color'] = np.where((graph_data.Player == 'Kobe Bryant'),'y', graph_data.color)

size = 100 * ((graph_data['PP75'] - (graph_data['PP75'].min() - 1)) / (graph_data['PP75'].max()+2 - (graph_data['PP75'].min() - 1)))
plt.figure(figsize=(20,12), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="3-16 FGA per 75", y="TS+", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='3-16 Feet Playoff Scoring Peaks (1997-2022) > 15 PP75 >2000 MP', xlabel='3-16 feet FGA per 75', ylabel='TS+')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Tim Duncan', markerfacecolor='#EF426F',  markersize=10),
                   Line2D([0], [0], marker='o', color='#EAEAF2', label='Kobe Bryant', markerfacecolor='y',  markersize=10)]
ax.legend(handles=legend_elements, loc='upper right')

# ridge regression
x_pred = graph_data['3-16 FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['TS+']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
ax.plot(x_pred, pred, color="k", linewidth=3)

graph_data = graph_data[(graph_data['Player'] != 'Tim Duncan')]
graph_data = graph_data[(graph_data['Player'] != 'Kobe Bryant')]
label_point(graph_data['3-16 FGA per 75'], graph_data['TS+'], graph_data['Player'], plt.gca())
plt.savefig(f'3-16_TS_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title 3-3P Graph (Peaks)
graph_data =  pd.read_csv('2P_Jumpshots_Peaks.csv', encoding='utf8')
graph_data = graph_data[(graph_data['MP'] >= 2000)]

graph_data = graph_data.sort_values('Player', ascending=False)

Tony = (graph_data['Player']=='Tim Duncan')
graph_data['color']= np.where(Tony==True , "#EF426F", "#00B2A9")
graph_data['color'] = np.where((graph_data.Player == 'Kobe Bryant'),'y', graph_data.color)

size = 100 * ((graph_data['PP75'] - (graph_data['PP75'].min() - 1)) / (graph_data['PP75'].max()+2 - (graph_data['PP75'].min() - 1)))
plt.figure(figsize=(20,12), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="2P Jumpshot FGA per 75", y="2P Jumpshot PPS", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':0.7, 's':size*4, 'linewidth':0.9, 'edgecolor':'k'})
ax.set(title='2P Jumpshot Playoff Scoring Peaks (1997-2022) > 15 PP75 >2000 MP', xlabel='2P Jumpshot FGA per 75', ylabel='2P Jumpshot PPS')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Tim Duncan', markerfacecolor='#EF426F',  markersize=10),
                   Line2D([0], [0], marker='o', color='#EAEAF2', label='Kobe Bryant', markerfacecolor='y',  markersize=10)]
ax.legend(handles=legend_elements, loc='upper right')

# ridge regression
x_pred = graph_data['2P Jumpshot FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['2P Jumpshot PPS']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
ax.plot(x_pred, pred, color="k", linewidth=3)

graph_data = graph_data[(graph_data['Player'] != 'Tim Duncan')]
graph_data = graph_data[(graph_data['Player'] != 'Kobe Bryant')]
label_point(graph_data['2P Jumpshot FGA per 75'], graph_data['2P Jumpshot PPS'], graph_data['Player'], plt.gca())
plt.savefig(f'2P_PPS_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title 3-3P Graph TS+ (Peaks)
graph_data =  pd.read_csv('2P_Jumpshots_Peaks.csv', encoding='utf8')
graph_data = graph_data.sort_values('Player', ascending=False)
#graph_data = graph_data[(graph_data['Player'] != 'Chris Paul')]

Tony = (graph_data['Player']=='Tim Duncan')
graph_data['color']= np.where(Tony==True , "#EF426F", "#00B2A9")
graph_data['color'] = np.where((graph_data.Player == 'Kobe Bryant'),'y', graph_data.color)

size = 100 * ((graph_data['PP75'] - (graph_data['PP75'].min() - 1)) / (graph_data['PP75'].max()+2 - (graph_data['PP75'].min() - 1)))
plt.figure(figsize=(20,12), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="2P Jumpshot FGA per 75", y="TS+", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':0.7, 's':size*4, 'linewidth':0.9, 'edgecolor':'k'})
ax.set(title='2P Jumpshot Playoff Scoring (1997-2022) Peaks > 15 PP75 >2000 MP', xlabel='2P Jumpshot FGA per 75', ylabel='TS+')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Tim Duncan', markerfacecolor='#EF426F',  markersize=10),
                   Line2D([0], [0], marker='o', color='#EAEAF2', label='Kobe Bryant', markerfacecolor='y',  markersize=10)]
ax.legend(handles=legend_elements, loc='upper right')

# ridge regression
x_pred = graph_data['2P Jumpshot FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['TS+']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
ax.plot(x_pred, pred, color="k", linewidth=3)

graph_data = graph_data[(graph_data['Player'] != 'Tim Duncan')]
graph_data = graph_data[(graph_data['Player'] != 'Kobe Bryant')]
label_point(graph_data['2P Jumpshot FGA per 75'], graph_data['TS+'], graph_data['Player'], plt.gca())
plt.savefig(f'2P_TS_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title 3-3P Graph PTS (Peaks)
graph_data =  pd.read_csv('2P_Jumpshots_Peaks.csv', encoding='utf8')
graph_data = graph_data.sort_values('Player', ascending=False)
#graph_data = graph_data[(graph_data['Player'] != 'Chris Paul')]

Tony = (graph_data['Player']=='Tim Duncan')
graph_data['color']= np.where(Tony==True , "#EF426F", "#00B2A9")
graph_data['color'] = np.where((graph_data.Player == 'Kobe Bryant'),'y', graph_data.color)

size = 100 * ((graph_data['MP'] - (graph_data['MP'].min() - 50)) / (graph_data['MP'].max() - (graph_data['MP'].min() - 50)))
plt.figure(figsize=(20,12), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="PP75", y="TS+", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':0.7, 's':size*4, 'linewidth':0.9, 'edgecolor':'k'})
ax.set(title='2P Jumpshot Playoff Scoring (1997-2022) Peaks > 15 PP75 >2000 MP', xlabel='PP75', ylabel='TS+')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Tim Duncan', markerfacecolor='#EF426F',  markersize=10),
                   Line2D([0], [0], marker='o', color='#EAEAF2', label='Kobe Bryant', markerfacecolor='y',  markersize=10)]
ax.legend(handles=legend_elements, loc='upper right')

# ridge regression
graph_data = graph_data[(graph_data['Player'] != 'Tim Duncan')]
graph_data = graph_data[(graph_data['Player'] != 'Kobe Bryant')]
label_point(graph_data['PP75'], graph_data['TS+'], graph_data['Player'], plt.gca())
plt.savefig(f'2P_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title mid edit
graph_data = pd.read_csv('/content/Midrange.csv', encoding='utf8')
graph_data = graph_data.drop(columns=['0-3 feet PPS',  '0-3 Freq',  '0-3 FGA per 75', '3-10 feet PPS',  '3-10 Freq',  '3-10 FGA per 75',
  '10-16 feet PPS',  '10-16 Freq', '10-16 FGA per 75', '16-3P feet PPS', '16-3P Freq',  '16-3P FGA per 75',  '3P PPS',  '3P Freq',  '3P FGA per 75'])
print(graph_data)
graph_data.to_csv('new_mid.csv', index=False)

#@title 2P jumpshot edit
graph_data = pd.read_csv('/content/2P_Jumpshots.csv', encoding='utf8')
graph_data = graph_data.drop(columns=['0-3 feet PPS',  '0-3 Freq',  '0-3 FGA per 75', '3-10 feet PPS',  '3-10 Freq',  '3-10 FGA per 75',
  '10-16 feet PPS',  '10-16 Freq', '10-16 FGA per 75', '16-3P feet PPS', '16-3P Freq',  '16-3P FGA per 75',  '3P PPS',  '3P Freq',  '3P FGA per 75'])
print(graph_data)
graph_data.to_csv('2P_Jumpshot_Min_4_FGA.csv', index=False)

graph_data['Jumpshot FGA per 75'] = graph_data['3-10 FGA per 75'] + graph_data['10-16 FGA per 75'] + graph_data['16-3P FGA per 75'] + + graph_data['3P FGA per 75']
total_freq = graph_data['3-10 Freq'] + graph_data['10-16 Freq'] + graph_data['16-3P Freq'] + graph_data['3P Freq']
graph_data['Jumpshot PPS'] = (((graph_data['3-10 Freq']) / total_freq) * graph_data['3-10 feet PPS']) + (((graph_data['10-16 Freq']) / total_freq) * graph_data['10-16 feet PPS']) + (((graph_data['16-3P Freq']) / total_freq) * graph_data['16-3P feet PPS']) + + (((graph_data['3P Freq']) / total_freq) * graph_data['3P PPS'])

In [ ]:
print_single_player_nba_shooting_data('Damian Lillard', f"https://www.basketball-reference.com/players/l/lillada01.html")

In [ ]:
#@title print_single_player_nba_shooting_data function

def print_single_player_nba_shooting_data(player, player_url):

    final_player_shooting_df = pd.DataFrame(columns = ['Lg', 'G', 'MP', 'FG%', 'Dist.',
                                                          '2Pfreq', '0-3freq', '3-10freq', '10-16freq', '16-3Pfreq', '3Pfreq', '2P', '0-3', '3-10',
                                                          '10-16', '16-3P', '3P', '2P', '3P',	'%FGAdunks',	'#dunks',	'%3PAcorner',
                                                          '3P%corner', 'Heave Att.', 'Heave Mad.'
                                                         ])

    wd = webdriver.Chrome('chromedriver',options=chrome_options)
    wd.get(player_url)
    html = wd.page_source
    soup = BeautifulSoup(html)

    for first_div in soup.find_all('div', attrs={'id': 'all_shooting-playoffs_shooting'}):
      second_div = first_div.find('div', attrs={'id': 'switcher_shooting-playoffs_shooting'})
      first_table = second_div.find('table', attrs={'id': 'playoffs_shooting'})
      foot = first_table.find('tfoot')

    rows = foot.find('tr')
    player_shooting_distribution_stats = [[td.getText() for td in rows.findAll('td')]]

    # remove empty rows
    player_shooting_distribution_stats = [e for e in player_shooting_distribution_stats if e != []]

    new_list = [[s for s in sub_list if s] for sub_list in player_shooting_distribution_stats]

    headers = ['Lg', 'G', 'MP', 'FG%', 'Dist.',
               '2Pfreq', '0-3freq', '3-10freq', '10-16freq', '16-3Pfreq', '3Pfreq', '2P', '0-3', '3-10',
               '10-16', '16-3P', '3P', '2P', '3P',	'%FGAdunks',	'#dunks',	'%3PAcorner',
               '3P%corner', 'Heave Att.', 'Heave Mad.']

    each_year = pd.DataFrame(new_list, columns = headers)

    # append dataframe
    final_player_shooting_df = final_player_shooting_df.append(each_year)

    new_df = pd.DataFrame(columns = ['Player', 'Midrange PPS', 'Mid Freq', 'PP75', 'TS+', 'MP'])
    headers_list = ['Player', 'Midrange PPS', 'Mid Freq', 'PP75', 'TS+', 'MP']

    freq = float(final_player_shooting_df['10-16freq']) + float(final_player_shooting_df['16-3Pfreq'])


    pps = ((float(final_player_shooting_df['10-16freq']) / freq) * ( ( float(final_player_shooting_df['10-16'] )*2 ))) + ((float(final_player_shooting_df['16-3Pfreq']) / freq) * ( ( float(final_player_shooting_df['16-3P'] ) *2)))
    round(pps, 3)
    round(freq, 3)

    (player, midrange_PPS, mid_Freq, pp75, ts, mp) = playoff_career_scoring_output(player, pps, freq, import_player_since74playoffs_per75_df)
    new_df.loc[len(new_df)] = [player, midrange_PPS, mid_Freq, pp75, ts, mp]
    outfile = f"{player}_Playoff_Midrange_Data.csv"
    print(new_df)
    new_df.to_csv(outfile, index=False)
def playoff_career_scoring_output(player, pps, freq, postseason_data):
  player_data = postseason_data[(postseason_data.Player == player)]

  total_mp = 0
  total_pts = 0
  total_ts = 0
  mp_list = []

  # find total minutes a
  for row in player_data['MP']:
    mp_list.append(row)
    total_mp += row
  if total_mp == 0:
    total_mp = 1

  # find total PTS a
  i = 0
  for row in player_data['PTS']:
    total_pts += row * (mp_list[i] / total_mp)
    i = i + 1

  # find total TS+ a
  i = 0
  for row in player_data['TS%+']:
    total_ts += row * (mp_list[i] / total_mp)
    i = i + 1

  round(total_pts, 2)
  total_pts = '%.2f' % round(total_pts, 2)
  total_ts = '%.2f' % round(total_ts, 2)
  pps = '%.3f' % round(pps, 3)
  return (player, pps, freq, total_pts, total_ts, total_mp)

**PLOT SHOOTING DISTRIBUTION DATA CAREER**

In [ ]:
#@title Midrange Graph
graph_data =  pd.read_csv('/content/NBA_Playoff_all_shooting_data_df.csv', encoding='utf8')

graph_data['Mid FGA per 75'] = graph_data['10-16 FGA per 75'] + graph_data['16-3P FGA per 75']
graph_data = graph_data[(graph_data['MP'] >= 2000)]
total_freq = graph_data['10-16 Freq'] + graph_data['16-3P Freq']
graph_data['Midrange PPS'] = (((graph_data['10-16 Freq']) / total_freq) * graph_data['10-16 feet PPS']) + (((graph_data['16-3P Freq']) / total_freq) * graph_data['16-3P feet PPS'])
graph_data = graph_data.dropna()
graph_data = graph_data[(graph_data['Mid FGA per 75'] >= 2)]
graph_data = graph_data.sort_values('Player', ascending=False)

graph_data['color']= "#00B2A9"


size = 100 * ((graph_data['MP'] - (graph_data['MP'].min() - (graph_data['MP'].min()*.15))) / (graph_data['MP'].max() - (graph_data['MP'].min() - (graph_data['MP'].min()*.15))))
plt.figure(figsize=(17,9), tight_layout=True)
plt.ylim(0.55, 1.0)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="Mid FGA per 75", y="Midrange PPS", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':0.6, 's':size*4, 'linewidth':0.4, 'edgecolor':'k'})
ax.set(title='Playoff Midrange Shooting (1997-2022) >2000 MP', xlabel='Mid FGA per 75', ylabel='Mid PPS')


# ridge regression
x_pred = graph_data['Mid FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['Midrange PPS']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
#ax.plot(x_pred, pred, color="k", linewidth=3)
selected_dots = graph_data[((graph_data['Player'] == 'John Stockton') | (graph_data['Player'] == 'Seth Curry')
| (graph_data['Player'] == 'Chris Paul') | (graph_data['Player'] == 'Michael Jordan') | (graph_data['Player'] == 'Kevin Durant')
| (graph_data['Player'] == 'Dirk Nowitzki') | (graph_data['Player'] == 'Stephen Curry') | (graph_data['Player'] == 'Kobe Bryant')
| (graph_data['Player'] == 'Damian Lillard') | (graph_data['Player'] == 'Dennis Rodman')
| (graph_data['Player'] == 'DeMarre Carroll') | (graph_data['MP'] >= 5000) | (graph_data['Midrange PPS'] >= .9) | (graph_data['Mid FGA per 75'] >= 8) | ((graph_data['Midrange PPS'] <= .65) & (graph_data['Player'] != "Marcus Camby")))]
label_point(selected_dots['Mid FGA per 75'], selected_dots['Midrange PPS'], selected_dots['Player'], plt.gca())
plt.savefig(f'Midrange_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title Manu/Tony Midrange Graph
graph_data =  pd.read_csv('/content/NBA_Playoff_all_shooting_data_df.csv', encoding='utf8')

graph_data['Mid FGA per 75'] = graph_data['10-16 FGA per 75'] + graph_data['16-3P FGA per 75']
graph_data = graph_data[(graph_data['MP'] >= 2000)]
total_freq = graph_data['10-16 Freq'] + graph_data['16-3P Freq']
graph_data['Midrange PPS'] = (((graph_data['10-16 Freq']) / total_freq) * graph_data['10-16 feet PPS']) + (((graph_data['16-3P Freq']) / total_freq) * graph_data['16-3P feet PPS'])
graph_data = graph_data.dropna()
graph_data = graph_data[(graph_data['Mid FGA per 75'] >= 2)]
graph_data = graph_data.sort_values('Player', ascending=False)

Tony = (graph_data['Player']=='Tony Parker')
graph_data['color']= np.where(Tony==True , "#EF426F", "#00B2A9")
graph_data['color'] = np.where((graph_data.Player == 'Manu Ginóbili'),'#FF8200', graph_data.color)

size = 100 * ((graph_data['MP'] - (graph_data['MP'].min() - (graph_data['MP'].min()*.15))) / (graph_data['MP'].max() - (graph_data['MP'].min() - (graph_data['MP'].min()*.15))))
plt.figure(figsize=(17,9), tight_layout=True)
plt.ylim(0.55, 1.0)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="Mid FGA per 75", y="Midrange PPS", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':0.6, 's':size*4, 'linewidth':0.4, 'edgecolor':'k'})
ax.set(title='Playoff Midrange Shooting (1997-2022) >2000 MP', xlabel='Mid FGA per 75', ylabel='Mid PPS')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Tony Parker', markerfacecolor='#EF426F',  markersize=10),
                   Line2D([0], [0], marker='o', color='#EAEAF2', label='Manu Ginóbili', markerfacecolor='#FF8200',  markersize=10)]
ax.legend(handles=legend_elements, loc='upper right')


# ridge regression
x_pred = graph_data['Mid FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['Midrange PPS']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
#ax.plot(x_pred, pred, color="k", linewidth=3)
selected_dots = graph_data[(graph_data['Player'] != 'Tony Parker') & (graph_data['Player'] != 'Manu Ginóbili') & ((graph_data['Player'] == 'John Stockton') | (graph_data['Player'] == 'Seth Curry')
| (graph_data['Player'] == 'Chris Paul') | (graph_data['Player'] == 'Michael Jordan') | (graph_data['Player'] == 'Kevin Durant')
| (graph_data['Player'] == 'Dirk Nowitzki') | (graph_data['Player'] == 'Stephen Curry') | (graph_data['Player'] == 'Kobe Bryant')
| (graph_data['Player'] == 'Damian Lillard') | (graph_data['Player'] == 'Dennis Rodman')
| (graph_data['Player'] == 'DeMarre Carroll') | (graph_data['MP'] >= 5000) | (graph_data['Midrange PPS'] >= .9) | (graph_data['Mid FGA per 75'] >= 8) | ((graph_data['Midrange PPS'] <= .65) & (graph_data['Player'] != "Marcus Camby")))]
label_point(selected_dots['Mid FGA per 75'], selected_dots['Midrange PPS'], selected_dots['Player'], plt.gca())
plt.savefig(f'Midrange_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title demar Midrange Graph
graph_data =  pd.read_csv('/content/NBA_Playoff_all_shooting_data_df.csv', encoding='utf8')
graph_data = graph_data[(graph_data['MP'] >= 2000)]

graph_data['Mid FGA per 75'] = graph_data['10-16 FGA per 75'] + graph_data['16-3P FGA per 75']
graph_data = graph_data[(graph_data['MP'] >= 1000)]
total_freq = graph_data['10-16 Freq'] + graph_data['16-3P Freq']
graph_data['Midrange PPS'] = (((graph_data['10-16 Freq']) / total_freq) * graph_data['10-16 feet PPS']) + (((graph_data['16-3P Freq']) / total_freq) * graph_data['16-3P feet PPS'])
graph_data = graph_data.dropna()
graph_data = graph_data[(graph_data['Mid FGA per 75'] >= 1)]
graph_data = graph_data.sort_values('Player', ascending=False)

Tony = (graph_data['Player']=='Damian Lillard')
graph_data['color']= np.where(Tony==True , "r", "#00B2A9")
graph_data['color'] = np.where((graph_data.Player == 'Shaquille O\'Neal'),'y', graph_data.color)

size = 100 * ((graph_data['MP'] - (graph_data['MP'].min() - 50)) / (graph_data['MP'].max() - (graph_data['MP'].min() - 50)))
plt.figure(figsize=(17,9), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="Mid FGA per 75", y="Midrange PPS", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='Playoff Midrange Shooting (1997-2022) >2000 MP', xlabel='Mid FGA per 75', ylabel='Mid PPS')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Damian Lillard', markerfacecolor='r',  markersize=10),
                   Line2D([0], [0], marker='o', color='#EAEAF2', label='Shaquille O\'Neal', markerfacecolor='y',  markersize=10)]
ax.legend(handles=legend_elements, loc='upper right')


# ridge regression
x_pred = graph_data['Mid FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['Midrange PPS']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
#ax.plot(x_pred, pred, color="", linewidth=3)
graph_data = graph_data[(graph_data['Player'] != 'Shaquille O\'Neal')]
selected_dots = graph_data[(graph_data['Player'] == 'John Stockton') | (graph_data['Player'] == 'Seth Curry')
| (graph_data['Player'] == 'Chris Paul') | (graph_data['Player'] == 'Michael Jordan') | (graph_data['Player'] == 'Kevin Durant')
| (graph_data['Player'] == 'Dirk Nowitzki') | (graph_data['Player'] == 'Stephen Curry') | (graph_data['Player'] == 'Kobe Bryant') | (graph_data['Player'] == 'Dennis Rodman')
| (graph_data['Player'] == 'DeMarre Carroll') | (graph_data['MP'] >= 6000)]
label_point(selected_dots['Mid FGA per 75'], selected_dots['Midrange PPS'], selected_dots['Player'], plt.gca())
plt.savefig(f'Midrange_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title demar 3-3P Graph
graph_data =  pd.read_csv('/content/NBA_Playoff_all_shooting_data_df.csv', encoding='utf8')
graph_data = graph_data[(graph_data['MP'] >= 2000)]

graph_data['2P Jumpshot FGA per 75'] = graph_data['3-10 FGA per 75'] + graph_data['10-16 FGA per 75'] + graph_data['16-3P FGA per 75']
total_freq = graph_data['3-10 Freq'] + graph_data['10-16 Freq'] + graph_data['16-3P Freq']
graph_data['2P Jumpshot PPS'] = (((graph_data['3-10 Freq']) / total_freq) * graph_data['3-10 feet PPS']) + (((graph_data['10-16 Freq']) / total_freq) * graph_data['10-16 feet PPS']) + (((graph_data['16-3P Freq']) / total_freq) * graph_data['16-3P feet PPS'])

graph_data = graph_data[(graph_data['2P Jumpshot FGA per 75'] >= 4)]
graph_data = graph_data[(graph_data['PP75'] >= 15)]
graph_data = graph_data.sort_values('Player', ascending=False)

Tony = (graph_data['Player']=='DeMar DeRozan')
graph_data['color']= np.where(Tony==True , "r", "#00B2A9")

size = 100 * ((graph_data['PP75'] - graph_data['PP75'].min()) / (graph_data['PP75'].max() - graph_data['PP75'].min()))
plt.figure(figsize=(20,12), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="2P Jumpshot FGA per 75", y="2P Jumpshot PPS", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='3 Feet - 3P Line Playoff Scoring (1997-2022) > 15 PP75 >2000 MP', xlabel='2P Jumpshot FGA per 75', ylabel='2P Jumpshot PPS')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='DeMar DeRozan', markerfacecolor='r',  markersize=10)]
ax.legend(handles=legend_elements, loc='upper right')

# ridge regression
x_pred = graph_data['2P Jumpshot FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['2P Jumpshot PPS']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
ax.plot(x_pred, pred, color="k", linewidth=3)

graph_data = graph_data[(graph_data['Player'] != 'DeMar DeRozan')]
label_point(graph_data['2P Jumpshot FGA per 75'], graph_data['2P Jumpshot PPS'], graph_data['Player'], plt.gca())
plt.savefig(f'2P_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title 0-3 Graph
graph_data =  pd.read_csv('/content/NBA_Playoff_all_shooting_data_df.csv', encoding='utf8')
graph_data = graph_data[(graph_data['0-3 FGA per 75'] >= 4)]

graph_data = graph_data.sort_values('Player', ascending=False)

graph_data = graph_data[(graph_data['Player'] == 'Tony Parker')  | (graph_data['Player'] == 'Tony Allen') | (graph_data['Player'] == 'Derrick Rose') | (graph_data['Player'] == 'Russell Westbrook')
| (graph_data['Player'] == 'Stephen Curry') | (graph_data['Player'] == 'Dwyane Wade') | (graph_data['Player'] == 'John Wall') | (graph_data['Player'] == 'Rajon Rondo')
| (graph_data['Player'] == 'Tracy McGrady') | (graph_data['Player'] == 'Dennis Schröder') | (graph_data['Player'] == 'Allen Iverson') | (graph_data['Player'] == 'Antoine Walker')
| (graph_data['Player'] == 'Baron Davis') | (graph_data['Player'] == 'Andre Miller') | (graph_data['Player'] == 'Michael Jordan') | (graph_data['Player'] == 'Donovan Mitchell')
| (graph_data['Player'] == 'Vince Carter') | (graph_data['Player'] == 'Damian Lillard') | (graph_data['Player'] == 'Leandro Barbosa')
| (graph_data['Player'] == 'Gary Payton') | (graph_data['Player'] == 'Avery Johnson') | (graph_data['Player'] == 'Jeff Teague')
| (graph_data['Player'] == 'Kyrie Irving') | (graph_data['Player'] == 'Shandon Anderson') | (graph_data['Player'] == 'Norman Powell')
| (graph_data['Player'] == 'Monta Ellis') | (graph_data['Player'] == 'Kobe Bryant') | (graph_data['Player'] == 'Eric Bledsoe')
| (graph_data['Player'] == 'Avery Bradley') | (graph_data['Player'] == 'Jimmy Butler')  | (graph_data['Player'] == 'Bobby Jackson')
| (graph_data['Player'] == 'Richard Hamilton') | (graph_data['Player'] == 'Ronald Murray') | (graph_data['Player'] == 'Jameer Nelson')
| (graph_data['Player'] == 'Kerry Kittles') | (graph_data['Player'] == 'Jaylen Brown') | (graph_data['Player'] == 'Deron Williams')
| (graph_data['Player'] == 'James Harden') | (graph_data['Player'] == 'Lance Stephenson') | (graph_data['Player'] == 'Manu Ginóbili')
| (graph_data['Player'] == 'Goran Dragić')]

Tony = (graph_data['Player']=='Tony Parker')
graph_data['color']= np.where(Tony==True , "#EF426F", "#00B2A9")
graph_data['color'] = np.where((graph_data.Player == 'Manu Ginóbili'),'#FF8200', graph_data.color)

size = 100 * ((graph_data['MP'] - graph_data['MP'].min()) / (graph_data['MP'].max() - graph_data['MP'].min()))
plt.figure(figsize=(17,9), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="0-3 FGA per 75", y="0-3 feet PPS", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='0-3 Feet Playoff Scoring (1997-2022) >1000 MP', xlabel='0-3 Feet FGA per 75', ylabel='0-3 Feet PPS')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Tony Parker', markerfacecolor='#EF426F',  markersize=10),
                   Line2D([0], [0], marker='o', color='#EAEAF2', label='Manu Ginóbili', markerfacecolor='#FF8200',  markersize=10)]
ax.legend(handles=legend_elements, loc='upper right')

# ridge regression
x_pred = graph_data['0-3 FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['0-3 feet PPS']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
graph_data = graph_data[(graph_data['Player'] != 'Tony Parker')]
graph_data = graph_data[(graph_data['Player'] != 'Manu Ginóbili')]
graph_data = graph_data[(graph_data['Player'] != 'Norman Powell')]
graph_data = graph_data[(graph_data['Player'] != 'Ronald Murray')]
label_point(graph_data['0-3 FGA per 75'], graph_data['0-3 feet PPS'], graph_data['Player'], plt.gca())
plt.savefig(f'0-3_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title 0-3 Graph Tim
graph_data =  pd.read_csv('/content/NBA_Playoff_all_shooting_data_df.csv', encoding='utf8')
graph_data = graph_data[(graph_data['0-3 FGA per 75'] >= 4)]
graph_data = graph_data[(graph_data['MP'] >= 2000)]
graph_data = graph_data[(graph_data['PP75'] >= 15)]
graph_data = graph_data.sort_values('Player', ascending=False)

Tony = (graph_data['Player']=='Tim Duncan')
graph_data['color']= np.where(Tony==True , "#EF426F", "#00B2A9")
graph_data['color'] = np.where((graph_data.Player == 'Kobe Bryant'),'y', graph_data.color)

size = 100 * ((graph_data['PP75'] - graph_data['PP75'].min()) / (graph_data['PP75'].max() - graph_data['PP75'].min()))
plt.figure(figsize=(20,12), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="0-3 FGA per 75", y="0-3 feet PPS", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='0-3 Feet Playoff Scoring (1997-2022) > 15 PP75 >2000 MP', xlabel='0-3 Feet FGA per 75', ylabel='0-3 Feet PPS')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Tim Duncan', markerfacecolor='#EF426F',  markersize=10),
                   Line2D([0], [0], marker='o', color='#EAEAF2', label='Kobe Bryant', markerfacecolor='y',  markersize=10)]
ax.legend(handles=legend_elements, loc='lower right')

# ridge regression
x_pred = graph_data['0-3 FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['0-3 feet PPS']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
ax.plot(x_pred, pred, color="k", linewidth=3)

graph_data = graph_data[(graph_data['Player'] != 'Tim Duncan')]
graph_data = graph_data[(graph_data['Player'] != 'Kobe Bryant')]
label_point(graph_data['0-3 FGA per 75'], graph_data['0-3 feet PPS'], graph_data['Player'], plt.gca())
plt.savefig(f'0-3_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title 0-3 Graph Tim TS+
graph_data =  pd.read_csv('/content/NBA_Playoff_all_shooting_data_df.csv', encoding='utf8')
graph_data = graph_data[(graph_data['0-3 FGA per 75'] >= 4)]
graph_data = graph_data[(graph_data['MP'] >= 2000)]
graph_data = graph_data[(graph_data['PP75'] >= 15)]
graph_data = graph_data.sort_values('Player', ascending=False)

Tony = (graph_data['Player']=='Tim Duncan')
graph_data['color']= np.where(Tony==True , "#EF426F", "#00B2A9")
graph_data['color'] = np.where((graph_data.Player == 'Kobe Bryant'),'y', graph_data.color)

size = 100 * ((graph_data['PP75'] - graph_data['PP75'].min()) / (graph_data['PP75'].max() - graph_data['PP75'].min()))
plt.figure(figsize=(20,12), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="0-3 FGA per 75", y="TS+", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='0-3 Feet Playoff Scoring (1997-2022) > 15 PP75 >2000 MP', xlabel='0-3 Feet FGA per 75', ylabel='TS+')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Tim Duncan', markerfacecolor='#EF426F',  markersize=10),
                   Line2D([0], [0], marker='o', color='#EAEAF2', label='Kobe Bryant', markerfacecolor='y',  markersize=10)]
ax.legend(handles=legend_elements, loc='lower right')

# ridge regression
x_pred = graph_data['0-3 FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['TS+']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
ax.plot(x_pred, pred, color="k", linewidth=3)

graph_data = graph_data[(graph_data['Player'] != 'Tim Duncan')]
graph_data = graph_data[(graph_data['Player'] != 'Kobe Bryant')]
label_point(graph_data['0-3 FGA per 75'], graph_data['TS+'], graph_data['Player'], plt.gca())
plt.savefig(f'0-3_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title 3-3P Graph
graph_data =  pd.read_csv('/content/NBA_Playoff_all_shooting_data_df.csv', encoding='utf8')
graph_data = graph_data[(graph_data['MP'] >= 2000)]

graph_data['2P Jumpshot FGA per 75'] = graph_data['3-10 FGA per 75'] + graph_data['10-16 FGA per 75'] + graph_data['16-3P FGA per 75']
total_freq = graph_data['3-10 Freq'] + graph_data['10-16 Freq'] + graph_data['16-3P Freq']
graph_data['2P Jumpshot PPS'] = (((graph_data['3-10 Freq']) / total_freq) * graph_data['3-10 feet PPS']) + (((graph_data['10-16 Freq']) / total_freq) * graph_data['10-16 feet PPS']) + (((graph_data['16-3P Freq']) / total_freq) * graph_data['16-3P feet PPS'])

graph_data = graph_data[(graph_data['2P Jumpshot FGA per 75'] >= 4)]
graph_data = graph_data[(graph_data['PP75'] >= 15)]
graph_data = graph_data.sort_values('Player', ascending=False)

Tony = (graph_data['Player']=='Tim Duncan')
graph_data['color']= np.where(Tony==True , "#EF426F", "#00B2A9")
graph_data['color'] = np.where((graph_data.Player == 'Kobe Bryant'),'y', graph_data.color)

size = 100 * ((graph_data['PP75'] - graph_data['PP75'].min()) / (graph_data['PP75'].max() - graph_data['PP75'].min()))
plt.figure(figsize=(20,12), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="2P Jumpshot FGA per 75", y="2P Jumpshot PPS", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='2P Jumpshot Playoff Scoring (1997-2022) > 15 PP75 >2000 MP', xlabel='2P Jumpshot FGA per 75', ylabel='2P Jumpshot PPS')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Tim Duncan', markerfacecolor='#EF426F',  markersize=10),
                   Line2D([0], [0], marker='o', color='#EAEAF2', label='Kobe Bryant', markerfacecolor='y',  markersize=10)]
ax.legend(handles=legend_elements, loc='upper right')

# ridge regression
x_pred = graph_data['2P Jumpshot FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['2P Jumpshot PPS']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
ax.plot(x_pred, pred, color="k", linewidth=3)

graph_data = graph_data[(graph_data['Player'] != 'Tim Duncan')]
graph_data = graph_data[(graph_data['Player'] != 'Kobe Bryant')]
label_point(graph_data['2P Jumpshot FGA per 75'], graph_data['2P Jumpshot PPS'], graph_data['Player'], plt.gca())
plt.savefig(f'2P_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title 3-3P Graph TS+
graph_data =  pd.read_csv('/content/NBA_Playoff_all_shooting_data_df.csv', encoding='utf8')
graph_data = graph_data[(graph_data['MP'] >= 2000)]

graph_data['2P Jumpshot FGA per 75'] = graph_data['3-10 FGA per 75'] + graph_data['10-16 FGA per 75'] + graph_data['16-3P FGA per 75']
total_freq = graph_data['3-10 Freq'] + graph_data['10-16 Freq'] + graph_data['16-3P Freq']
graph_data['2P Jumpshot PPS'] = (((graph_data['3-10 Freq']) / total_freq) * graph_data['3-10 feet PPS']) + (((graph_data['10-16 Freq']) / total_freq) * graph_data['10-16 feet PPS']) + (((graph_data['16-3P Freq']) / total_freq) * graph_data['16-3P feet PPS'])

graph_data = graph_data[(graph_data['2P Jumpshot FGA per 75'] >= 4)]
graph_data = graph_data[(graph_data['PP75'] >= 15)]
graph_data = graph_data.sort_values('Player', ascending=False)

graph_data = graph_data[(graph_data['Player'] != 'Chris Paul')]

Tony = (graph_data['Player']=='Tim Duncan')
graph_data['color']= np.where(Tony==True , "#EF426F", "#00B2A9")
graph_data['color'] = np.where((graph_data.Player == 'Kobe Bryant'),'y', graph_data.color)

size = 100 * ((graph_data['PP75'] - graph_data['PP75'].min()) / (graph_data['PP75'].max() - graph_data['PP75'].min()))
plt.figure(figsize=(20,12), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="2P Jumpshot FGA per 75", y="TS+", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='2P Jumpshot Playoff Scoring (1997-2022) > 15 PP75 >2000 MP', xlabel='2P Jumpshot FGA per 75', ylabel='TS+')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Tim Duncan', markerfacecolor='#EF426F',  markersize=10),
                   Line2D([0], [0], marker='o', color='#EAEAF2', label='Kobe Bryant', markerfacecolor='y',  markersize=10)]
ax.legend(handles=legend_elements, loc='upper right')

# ridge regression
x_pred = graph_data['2P Jumpshot FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['TS+']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
ax.plot(x_pred, pred, color="k", linewidth=3)

graph_data = graph_data[(graph_data['Player'] != 'Tim Duncan')]
graph_data = graph_data[(graph_data['Player'] != 'Kobe Bryant')]
label_point(graph_data['2P Jumpshot FGA per 75'], graph_data['TS+'], graph_data['Player'], plt.gca())
plt.savefig(f'2P_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title 3-3P Graph TS+
graph_data =  pd.read_csv('/content/NBA_Playoff_prime_shooting_data_df.csv', encoding='utf8')
graph_data = graph_data[(graph_data['MP'] >= 2000)]

graph_data = graph_data[(graph_data['PP75'] >= 15)]
graph_data = graph_data.sort_values('Player', ascending=False)

graph_data = graph_data[(graph_data['Player'] != 'Chris Paul')]

Tony = (graph_data['Player']=='Tim Duncan 98 - 07')
graph_data['color']= np.where(Tony==True , "#EF426F", "#00B2A9")
graph_data['color'] = np.where((graph_data.Player == 'Kobe Bryant'),'y', graph_data.color)

size = 100 * ((graph_data['PP75'] - graph_data['PP75'].min()) / (graph_data['PP75'].max() - graph_data['PP75'].min()))
plt.figure(figsize=(20,12), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="3-10 FGA per 75", y="TS+", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='2P Jumpshot Playoff Scoring (1997-2022) > 15 PP75 >2000 MP', xlabel='3-10 FGA per 75', ylabel='TS+')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Tim Duncan', markerfacecolor='#EF426F',  markersize=10),
                   Line2D([0], [0], marker='o', color='#EAEAF2', label='Kobe Bryant', markerfacecolor='y',  markersize=10)]
ax.legend(handles=legend_elements, loc='upper right')

# ridge regression
x_pred = graph_data['3-10 FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['TS+']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
ax.plot(x_pred, pred, color="k", linewidth=3)

graph_data = graph_data[(graph_data['Player'] != 'Tim Duncan')]
graph_data = graph_data[(graph_data['Player'] != 'Kobe Bryant')]
label_point(graph_data['3-10 FGA per 75'], graph_data['TS+'], graph_data['Player'], plt.gca())
plt.savefig(f'2P_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title 3-16 PPS Graph
graph_data =  pd.read_csv('/content/NBA_Playoff_all_shooting_data_df.csv', encoding='utf8')

graph_data['3-16 FGA per 75'] = graph_data['3-10 FGA per 75'] + graph_data['10-16 FGA per 75']
total_freq = graph_data['3-10 Freq'] + graph_data['10-16 Freq']
graph_data['3-16 feet PPS'] = (((graph_data['3-10 Freq']) / total_freq) * graph_data['3-10 feet PPS']) + (((graph_data['10-16 Freq']) / total_freq) * graph_data['10-16 feet PPS'])

graph_data = graph_data.sort_values('Player', ascending=False)
graph_data = graph_data[(graph_data['MP'] >= 2000)]
graph_data = graph_data[(graph_data['PP75'] >= 15)]

Tony = (graph_data['Player']=='Tim Duncan')
graph_data['color']= np.where(Tony==True , "#EF426F", "#00B2A9")
graph_data['color'] = np.where((graph_data.Player == 'Kobe Bryant'),'y', graph_data.color)

size = 100 * ((graph_data['PP75'] - graph_data['PP75'].min()) / (graph_data['PP75'].max() - graph_data['PP75'].min()))
plt.figure(figsize=(20,12), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="3-16 FGA per 75", y="3-16 feet PPS", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='3-16 Feet Playoff Scoring (1997-2022) > 15 PP75 >2000 MP', xlabel='3-16 feet FGA per 75', ylabel='3-16 feet PPS')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Tim Duncan', markerfacecolor='#EF426F',  markersize=10),
                   Line2D([0], [0], marker='o', color='#EAEAF2', label='Kobe Bryant', markerfacecolor='y',  markersize=10)]
ax.legend(handles=legend_elements, loc='upper right')

# ridge regression
x_pred = graph_data['3-16 FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['3-16 feet PPS']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
ax.plot(x_pred, pred, color="k", linewidth=3)

graph_data = graph_data[(graph_data['Player'] != 'Tim Duncan')]
graph_data = graph_data[(graph_data['Player'] != 'Kobe Bryant')]
label_point(graph_data['3-16 FGA per 75'], graph_data['3-16 feet PPS'], graph_data['Player'], plt.gca())
plt.savefig(f'3-16_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title 3-16 TS+ Graph
graph_data =  pd.read_csv('/content/NBA_Playoff_all_shooting_data_df.csv', encoding='utf8')

graph_data['3-16 FGA per 75'] = graph_data['3-10 FGA per 75'] + graph_data['10-16 FGA per 75']
total_freq = graph_data['3-10 Freq'] + graph_data['10-16 Freq']
graph_data['3-16 feet PPS'] = (((graph_data['3-10 Freq']) / total_freq) * graph_data['3-10 feet PPS']) + (((graph_data['10-16 Freq']) / total_freq) * graph_data['10-16 feet PPS'])

graph_data = graph_data.sort_values('Player', ascending=False)
graph_data = graph_data[(graph_data['MP'] >= 2000)]
graph_data = graph_data[(graph_data['PP75'] >= 15)]

Tony = (graph_data['Player']=='Tim Duncan')
graph_data['color']= np.where(Tony==True , "#EF426F", "#00B2A9")
graph_data['color'] = np.where((graph_data.Player == 'Kobe Bryant'),'y', graph_data.color)

size = 100 * ((graph_data['PP75'] - graph_data['PP75'].min()) / (graph_data['PP75'].max() - graph_data['PP75'].min()))
plt.figure(figsize=(20,12), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="3-16 FGA per 75", y="TS+", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='3-16 Feet Playoff Scoring (1997-2022) > 15 PP75 >2000 MP', xlabel='3-16 feet FGA per 75', ylabel='TS+')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Tim Duncan', markerfacecolor='#EF426F',  markersize=10),
                   Line2D([0], [0], marker='o', color='#EAEAF2', label='Kobe Bryant', markerfacecolor='y',  markersize=10)]
ax.legend(handles=legend_elements, loc='upper right')

# ridge regression
x_pred = graph_data['3-16 FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['TS+']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
ax.plot(x_pred, pred, color="k", linewidth=3)

graph_data = graph_data[(graph_data['Player'] != 'Tim Duncan')]
graph_data = graph_data[(graph_data['Player'] != 'Kobe Bryant')]
label_point(graph_data['3-16 FGA per 75'], graph_data['TS+'], graph_data['Player'], plt.gca())
plt.savefig(f'3-16_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title 3-10 PPS Graph
graph_data =  pd.read_csv('/content/NBA_Playoff_all_shooting_data_df.csv', encoding='utf8')

graph_data = graph_data.sort_values('Player', ascending=False)
graph_data = graph_data[(graph_data['MP'] >= 2000)]
graph_data = graph_data[(graph_data['PP75'] >= 15)]

Tony = (graph_data['Player']=='Tim Duncan')
graph_data['color']= np.where(Tony==True , "#EF426F", "#00B2A9")
graph_data['color'] = np.where((graph_data.Player == 'Kobe Bryant'),'y', graph_data.color)

size = 100 * ((graph_data['PP75'] - graph_data['PP75'].min()) / (graph_data['PP75'].max() - graph_data['PP75'].min()))
plt.figure(figsize=(20,12), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="3-10 FGA per 75", y="3-10 feet PPS", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='3-10 Feet Playoff Scoring (1997-2022) > 15 PP75 >2000 MP', xlabel='3-10 FGA feet per 75', ylabel='3-10 feet PPS')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Tim Duncan', markerfacecolor='#EF426F',  markersize=10),
                   Line2D([0], [0], marker='o', color='#EAEAF2', label='Kobe Bryant', markerfacecolor='y',  markersize=10)]
ax.legend(handles=legend_elements, loc='upper right')

# ridge regression
x_pred = graph_data['3-10 FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['3-10 feet PPS']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
ax.plot(x_pred, pred, color="k", linewidth=3)

graph_data = graph_data[(graph_data['Player'] != 'Tim Duncan')]
graph_data = graph_data[(graph_data['Player'] != 'Kobe Bryant')]
label_point(graph_data['3-10 FGA per 75'], graph_data['3-10 feet PPS'], graph_data['Player'], plt.gca())
plt.savefig(f'3-10_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title 3-10 TS+ Graph
graph_data =  pd.read_csv('/content/NBA_Playoff_all_shooting_data_df.csv', encoding='utf8')


graph_data = graph_data.sort_values('Player', ascending=False)
graph_data = graph_data[(graph_data['MP'] >= 2000)]
graph_data = graph_data[(graph_data['PP75'] >= 15)]

Tony = (graph_data['Player']=='Tim Duncan')
graph_data['color']= np.where(Tony==True , "#EF426F", "#00B2A9")
graph_data['color'] = np.where((graph_data.Player == 'Kobe Bryant'),'y', graph_data.color)

size = 100 * ((graph_data['PP75'] - graph_data['PP75'].min()) / (graph_data['PP75'].max() - graph_data['PP75'].min()))
plt.figure(figsize=(20,12), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="3-10 FGA per 75", y="TS+", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='3-10 Feet Playoff Scoring (1997-2022) > 15 PP75 >2000 MP', xlabel='3-10 feet FGA per 75', ylabel='TS+')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Tim Duncan', markerfacecolor='#EF426F',  markersize=10),
                   Line2D([0], [0], marker='o', color='#EAEAF2', label='Kobe Bryant', markerfacecolor='y',  markersize=10)]
ax.legend(handles=legend_elements, loc='upper right')

# ridge regression
x_pred = graph_data['3-10 FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['TS+']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
ax.plot(x_pred, pred, color="k", linewidth=3)

graph_data = graph_data[(graph_data['Player'] != 'Tim Duncan')]
graph_data = graph_data[(graph_data['Player'] != 'Kobe Bryant')]
label_point(graph_data['3-10 FGA per 75'], graph_data['TS+'], graph_data['Player'], plt.gca())
plt.savefig(f'3-10_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title 0-10 PPS Graph
graph_data =  pd.read_csv('/content/NBA_Playoff_all_shooting_data_df.csv', encoding='utf8')

graph_data['0-10 FGA per 75'] = graph_data['3-10 FGA per 75'] + graph_data['0-3 FGA per 75']
total_freq = graph_data['3-10 Freq'] + graph_data['0-3 Freq']
graph_data['0-10 feet PPS'] = (((graph_data['3-10 Freq']) / total_freq) * graph_data['3-10 feet PPS']) + (((graph_data['0-3 Freq']) / total_freq) * graph_data['0-3 feet PPS'])

graph_data = graph_data.sort_values('Player', ascending=False)
graph_data = graph_data[(graph_data['MP'] >= 2000)]
graph_data = graph_data[(graph_data['PP75'] >= 15)]

Tony = (graph_data['Player']=='Tim Duncan')
graph_data['color']= np.where(Tony==True , "#EF426F", "#00B2A9")
graph_data['color'] = np.where((graph_data.Player == 'Kobe Bryant'),'y', graph_data.color)

size = 100 * ((graph_data['PP75'] - graph_data['PP75'].min()) / (graph_data['PP75'].max() - graph_data['PP75'].min()))
plt.figure(figsize=(20,12), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="0-10 FGA per 75", y="0-10 feet PPS", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='0-10 Feet Playoff Scoring (1997-2022) > 15 PP75 >2000 MP', xlabel='0-10 feet FGA per 75', ylabel='0-10 feet PPS')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Tim Duncan', markerfacecolor='#EF426F',  markersize=10),
                   Line2D([0], [0], marker='o', color='#EAEAF2', label='Kobe Bryant', markerfacecolor='y',  markersize=10)]
ax.legend(handles=legend_elements, loc='upper right')

# ridge regression
x_pred = graph_data['0-10 FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['0-10 feet PPS']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
ax.plot(x_pred, pred, color="k", linewidth=3)

graph_data = graph_data[(graph_data['Player'] != 'Tim Duncan')]
graph_data = graph_data[(graph_data['Player'] != 'Kobe Bryant')]
label_point(graph_data['0-10 FGA per 75'], graph_data['0-10 feet PPS'], graph_data['Player'], plt.gca())
plt.savefig(f'0-10_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title 0-10 TS+ Graph
graph_data =  pd.read_csv('/content/NBA_Playoff_all_shooting_data_df.csv', encoding='utf8')

graph_data['0-10 FGA per 75'] = graph_data['3-10 FGA per 75'] + graph_data['0-3 FGA per 75']
total_freq = graph_data['3-10 Freq'] + graph_data['0-3 Freq']
graph_data['0-10 feet PPS'] = (((graph_data['3-10 Freq']) / total_freq) * graph_data['3-10 feet PPS']) + (((graph_data['0-3 Freq']) / total_freq) * graph_data['0-3 feet PPS'])

graph_data = graph_data.sort_values('Player', ascending=False)
graph_data = graph_data[(graph_data['MP'] >= 2000)]
graph_data = graph_data[(graph_data['PP75'] >= 15)]

Tony = (graph_data['Player']=='Tim Duncan')
graph_data['color']= np.where(Tony==True , "#EF426F", "#00B2A9")
graph_data['color'] = np.where((graph_data.Player == 'Kobe Bryant'),'y', graph_data.color)

size = 100 * ((graph_data['PP75'] - graph_data['PP75'].min()) / (graph_data['PP75'].max() - graph_data['PP75'].min()))
plt.figure(figsize=(20,12), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="0-10 FGA per 75", y="TS+", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='0-10 Feet Playoff Scoring (1997-2022) > 15 PP75 >2000 MP', xlabel='0-10 feet FGA per 75', ylabel='TS+')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Tim Duncan', markerfacecolor='#EF426F',  markersize=10),
                   Line2D([0], [0], marker='o', color='#EAEAF2', label='Kobe Bryant', markerfacecolor='y',  markersize=10)]
ax.legend(handles=legend_elements, loc='upper right')

# ridge regression
x_pred = graph_data['0-10 FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['TS+']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
ax.plot(x_pred, pred, color="k", linewidth=3)

graph_data = graph_data[(graph_data['Player'] != 'Tim Duncan')]
graph_data = graph_data[(graph_data['Player'] != 'Kobe Bryant')]
label_point(graph_data['0-10 FGA per 75'], graph_data['TS+'], graph_data['Player'], plt.gca())
plt.savefig(f'0-10_Graph', bbox_inches='tight')
plt.show()